**Autor:** Enzo Bustos Da Silva

**Título:** Chatbot informativo multi-agentes

# Projeto Unificado - Chatbot Informativo

Ideia é fazer com que esse seja o documento que agrupa todos os módulos do chatbot informativo multi-agentes para encaminhar o projeto final da minha iniciação científica (que termina em janeiro).

Motivação são as Atas do Diário da Assembléia da República Portuguesa, quero justamente retirar os textos das falas e puxar informações úteis sobre elas, para isso quero criar um chatbot informativo que vai ser composto por diversos bots auxiliares e um bot agentes:







*   Bot Agente: Responsável por ser uma interface entre o usuário e a ferramenta, que irá coletar as informações principais, como qual Ata ou conjunto delas o usuário quer explorar (por seleção ou agrupamento temporal) e informações que os outros bots possam necessitar.

*   Bot de tópicos: A ideia principal é, cada uma das atas tem uma coluna (precisa ser revisada) que mostra qual item/itens de pauta estão sendo discutidos, se for um projeto parlamentar (lei, inquérito., petição...). A ideia é esse bot usar o BERTopic para extrair quais são os principais tópicos das conversas e englobá-los em grandes grupos como: Educação, Meio Ambiente ou Saúde e, associar os projetos de Lei a essas discussões.

* Bot de análise de sentimentos: Está meio no ar, mas a ideia principal seria usar análise de sentimentos para dizer se determinado partido está, ou não, sendo condizente com seu voto, dado uma fala. Ex.: Durante a discussão o parlamentar fez um discurso favorável e votou contra uma proposta. Acho que junto a isso, talvez seja mais fácil, mas com menos machine learning (precisa de dados de votações) tentar ter um bot que fale qual o partido político português que mais está próximo do usuário, podendo usar o mapa de coordenadas que já está feito e cerca de 20 a 30 questões que englobariam voto Contra, A Favor ou Abstenção para pautas quem mostram a face das ideias do partido (algo como legalização, aborto, pautas de certa forma gerais e algumas específicas).

* Bot de sumarização automática: Ideia é que dado um determinado item da pauta o bot resuma a discussão nos principais tópicos, para isso excluiria as falas de moderação do presidente e faria um resumo extrativo (apenas retirando sentenças mais importantes das discussões que aconteceram. Acho que não cabe sofisticar mais, mas poderia ter algum tipo de filtro duplo, por exemplo priorizar tópicos gerais sobre um dos temas do bot de tópicos, por exemplo Saúde, então o bot de sumarização faria um resumo das falas de Saúde de um conjunto de atas ou de uma ata, aliado a isso poderia ter um filtro sobre partidos também, então ao todo teria um filtro sobre o que um determinado partido falou sobre um dado tema X. 

In [ ]:
import pandas as pd
import torch
import time

df = pd.read_csv("dialog_lines_database_initiatives.csv")
df.drop(["Unnamed: 0", "Position"], axis=1, inplace=True)
#df.columns = ["Ata", "Data", "Pessoa", "Partido", "Texto", "Iniciativa"]
df[498:505]

,Transcript,Date,Person,Party,Text,Initiatives
498,DAR-002,18 DE SETEMBRO DE 2020,André Ventura,CH,"Sr. Presidente, Srs. Deputados: Temos de começ...","projeto de resolução n.º 471, projeto de lei n..."
499,DAR-002,18 DE SETEMBRO DE 2020,Presidente,Fernando Negrão,"Sr. Deputado, chamo a sua atenção para o tempo...","projeto de resolução n.º 471, projeto de lei n..."
500,DAR-002,18 DE SETEMBRO DE 2020,André Ventura,CH,"Vou terminar, Sr. Presidente. Portanto, não há...","projeto de resolução n.º 471, projeto de lei n..."
501,DAR-002,18 DE SETEMBRO DE 2020,Duarte Alves,PCP,Quando pagamos!… I SÉRIE — NÚMERO 2,"projeto de resolução n.º 471, projeto de lei n..."
502,DAR-002,18 DE SETEMBRO DE 2020,André Ventura,CH,Se pesquisarmos na internet por PCP e nacional...,"projeto de resolução n.º 471, projeto de lei n..."
503,DAR-002,18 DE SETEMBRO DE 2020,Presidente,Fernando Negrão,"Tem de terminar, Sr. Deputado.","projeto de resolução n.º 471, projeto de lei n..."
504,DAR-002,18 DE SETEMBRO DE 2020,André Ventura,CH,"Nós cá estamos para pagar, como sempre!","projeto de resolução n.º 471, projeto de lei n..."


# Sumarização Automática

Ideia: Como não existem bons modelos pré-treinados para português usaremos o M2M100 (Facebook) para traduzir para inglês, Pegasus (Google) para resumir e novamente o M2M100 para retornar ao português

M2M100: [HuggingFace](https://huggingface.co/facebook/m2m100_418M); [Paper](https://arxiv.org/pdf/2010.11125.pdf); [GitHub](https://github.com/pytorch/fairseq/tree/master/examples/m2m_100)

M2M100 parece fazer parte do Fairseq (Facebook AI Research Sequence-to-Sequence Toolkit)

Pegasus: [HuggingFace](https://huggingface.co/google/pegasus-xsum); [Paper](https://arxiv.org/pdf/1912.08777.pdf); [GitHub](https://github.com/google-research/pegasus)

In [25]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 28.0 MB/s 
     |████████████████████████████████| 895 kB 53.4 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 636 kB 50.7 MB/s 
     |████████████████████████████████| 3.3 MB 46.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [26]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 31.6 MB/s 


## Teste na 1a Tradução

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

m2m100_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="pt", tgt_lang="en")
m2m100_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

Downloading:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/272 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/908 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

### Teste com frase do DataFrame

In [ ]:
df_text = df["Text"].iloc[217]

print("Número de caracteres:", len(df_text))
print()
df_text

Número de caracteres: 1846



'Sr. Presidente, Sr.as e Srs. Deputados: No final da passada Sessão Legislativa, também como declaração política, apresentei aqui a disponibilidade do CDS para contribuir, ora em escrutínio, ora em complemento, para a urgente recuperação económica do País no período pandémico. Aqui estamos, Srs. Deputados, e é com pesar que não encontro essa disponibilidade no Governo, a quem cabem, naturalmente, as responsabilidades primordiais dessa mesma missão. Li e escutei atentamente a apresentação da dita Visão Estratégica para o Plano de Recuperação que aqui hoje debatemos. E em todo esse processo, lembrei-me de uma velha regra socialista: os socialistas, quando não querem falar do que não fizeram, lançam grandes debates daquilo que nunca farão. Além da já habitual tendência para pensar em forma de década, dei também conta da necessidade do Eng.º Costa Silva de apresentar os mais de 1000 contributos que recebeu dos cidadãos portugueses com uma moldura ideológica que pertence mais ao PS do que a

In [ ]:
m2m100_tokenizer.src_lang = "pt"

start = time.time()

encoded = m2m100_tokenizer(df_text, return_tensors="pt")
generated_tokens = m2m100_model.generate(**encoded, forced_bos_token_id=m2m100_tokenizer.get_lang_id("en"))
df_translated = m2m100_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

end = time.time()

print("Tempo de Execução:", str(round(end - start, 4)) + "s")

Tempo de Execução: 73.1865s


In [ ]:
#Não ficou bom o suficiente, resumiu por conta do M2M100, mas perdeu algumas informações
#Queremos apenas traduzir nessa parte

print("Número de caracteres:", len(df_translated[0]))
print()
df_translated[0]

Número de caracteres: 810



'Mr. President, Mr. Deputies, here we are, Mr. Deputies, and it is with regret that I do not find that availability in the Government, to whom the House of Representatives are, of course, the primary responsibilities of this same mission. I read and listened carefully to the presentation of the said Strategic Vision for the Recovery Plan that we discuss here today. And in all this process, I remember a old socialist rule for the urgent economic recovery of the country in the pandemic period. Here we are, Mr. Deputies, and it is with regret that I do not find that availability in the Government, to whom the House of Representatives are, of course, the primary responsibilities of this same mission. I read and listened carefully to the presentation of the said Strategic Vision for the Recovery Plan that'

In [ ]:
df_sentences = [
             "Sr. Presidente, Sr.as e Srs. Deputados: No final da passada Sessão Legislativa, também como declaração política, apresentei aqui a disponibilidade do CDS para contribuir, ora em escrutínio, ora em complemento, para a urgente recuperação económica do País no período pandémico.",
             "Aqui estamos, Srs. Deputados, e é com pesar que não encontro essa disponibilidade no Governo, a quem cabem, naturalmente, as responsabilidades primordiais dessa mesma missão.",
             "Li e escutei atentamente a apresentação da dita Visão Estratégica para o Plano de Recuperação que aqui hoje debatemos.",
             "E em todo esse processo, lembrei-me de uma velha regra socialista: os socialistas, quando não querem falar do que não fizeram, lançam grandes debates daquilo que nunca farão.",
             "Além da já habitual tendência para pensar em forma de década, dei também conta da necessidade do Eng.º Costa Silva de apresentar os mais de 1000 contributos que recebeu dos cidadãos portugueses com uma moldura ideológica que pertence mais ao PS do que a esses mesmos cidadãos: «uma derrota histórica das ideias ultraliberais», disse o Sr. Engenheiro.", 
             "Achei, Sr.as e Srs. Deputados, verdadeiramente extraordinário que um plano que se propõe pensar a 10 anos ainda venha embrulhado na retórica da crise anterior, felizmente vencida, como deve ser, com o esforço de todos os portugueses.", 
             "Honestamente, não me recordo de ouvir, nesta Câmara, uma única voz defender que a solução para vencer esta nova crise passe por anular o papel do Estado na economia ou na sociedade portuguesa, como, aliás, não me recordo de ouvir em nenhuma das anteriores legislaturas.", 
             "É estranho que, perante a dimensão dos desafios que hoje enfrentamos, os Srs. Membros do Governo e demais consultores insistam em preferir a retórica à realidade, o remoque ideológico ao contributo sustentado ou o fogo de vista à visão concreta.", 
]

print("Número de caracteres:", len(" ".join(df_sentences)))

Número de caracteres: 1846


In [ ]:
start = time.time()

encoded = list(map(lambda x: m2m100_tokenizer(x, return_tensors="pt"), df_sentences))
generated_tokens = list(map(lambda x: m2m100_model.generate(**x, forced_bos_token_id=m2m100_tokenizer.get_lang_id("en")), encoded))
translated = list(map(lambda x: m2m100_tokenizer.batch_decode(x, skip_special_tokens=True), generated_tokens))

end = time.time()

print("Tempo de Execução:", str(round(end - start, 4)) + "s")

Tempo de Execução: 134.2305s


In [ ]:
flat_translated = [sentence for embed_sentence in translated for sentence in embed_sentence]
df_translated = " ".join(flat_translated)

print("Número de caracteres:", len(df_translated))
print()
df_translated

Número de caracteres: 1917



'Mr. President, Mr. and Mr. Deputies: At the end of the last Legislative Session, also as a political statement, I presented here the availability of the CDS to contribute, now in scrutiny, now in complement, to the urgent economic recovery of the country in the pandemic period. Here we are, Mr. Deputies, and it is with regret that I do not find that availability in the Government, to whom, of course, the primary responsibilities of this same mission belong. I read and listened carefully to the presentation of the said Strategic Vision for the Recovery Plan that we are discussing here today. And in all this process, I remember a old socialist rule: when the socialists don’t want to talk about what they haven’t done, they launch great debates about what they’ll never do. In addition to the already usual tendency to think in the form of a decade, I also noticed the need of Eng.o Costa Silva to present the more than 1000 contributions he received from Portuguese citizens with an ideologic

Problemas principais:

- Tokenização de sentenças: Utilizar diretamente não é uma boa alternativa pelas características de sumarizador do M2M100
- Tempo de execução: achar uma outra alternativa para fazer em tempo real, já que uma discussão pode ter de 30 ~ 55 falas (longas) diferentes para serem processadas

### Teste com frase muito longa do DF

In [ ]:
#Maior texto que menciona corrupção
# https://debates.parlamento.pt/catalogo/r3/dar/01/14/02/041/2021-01-20/41
#text = df[df["Text"].str.contains("corrup")].reset_index().iloc[74]["Text"]

df = pd.read_csv("/content/dialog_lines_database_initiatives.csv")
long_text = df.iloc[9596]["Text"] + " " + df.iloc[9598]["Text"]
long_text = long_text.replace(" I SÉRIE — NÚMERO 41", "")
long_text

'Sr. Presidente, Sr.as e Srs. Deputados: O debate que nos é convocado aparenta ser um debate sobre o futebol e o impacto negativo do futebol, mas, na realidade, não é. Até vem à memória a frase do famoso treinador do Liverpool, Bill Shankly, que dizia: «O futebol não é uma questão de vida ou de morte, é muito mais importante do que isso». Na realidade, transpondo essa frase para este debate, o que deveríamos ter era um debate sobre incompatibilidades, impedimentos e conflito de interesses, que não é um debate de vida ou de morte, mas, na verdade, é muito mais importante do que isso, porque é um debate sobre a instituição, as regras que a regem e a forma como os mandatos são exercidos. Portanto, em primeiro lugar, este não é um debate sobre o que achamos ou deixamos de achar sobre o futebol, o mundo do futebol, os seus dirigentes, o que aí gostamos ou não gostamos. Portanto, não se trata de saber se vemos poesia num golo de Eusébio, se vemos brilhantismo numa tática dos Cinco Violinos o

In [ ]:
long_txt = "Sr. Presidente, Sr.as e Srs. Deputados: O debate que nos é convocado aparenta ser um debate sobre o futebol e o impacto negativo do futebol, mas, na realidade, não é. Até vem à memória a frase do famoso treinador do Liverpool, Bill Shankly, que dizia: «O futebol não é uma questão de vida ou de morte, é muito mais importante do que isso». Na realidade, transpondo essa frase para este debate, o que deveríamos ter era um debate sobre incompatibilidades, impedimentos e conflito de interesses, que não é um debate de vida ou de morte, mas, na verdade, é muito mais importante do que isso, porque é um debate sobre a instituição, as regras que a regem e a forma como os mandatos são exercidos. Portanto, em primeiro lugar, este não é um debate sobre o que achamos ou deixamos de achar sobre o futebol, o mundo do futebol, os seus dirigentes, o que aí gostamos ou não gostamos. Portanto, não se trata de saber se vemos poesia num golo de Eusébio, se vemos brilhantismo numa tática dos Cinco Violinos ou se ficamos sem ar quando Vítor Baía faz uma defesa. O ponto não é esse, e o debate não deve ser sobre futebol. Se o transformamos num debate sobre futebol, retiramos o debate de onde ele deve estar. Também sabemos — e o Sr. Deputado André Silva disse uma coisa com a qual é possível concordar — que hoje há um debate envenenado na opinião pública, e há programas televisivos que envenenam a forma como os cidadãos se relacionam uns com os outros, que instigam ao ódio, instigam à diferença entre os outros e que é aproveitado politicamente por alguns, e que, por isso, deve ser denunciado. Mas da denúncia à proibição vai um caminho bastante diferente. Portanto, ao contrário do que parece ser entendido, e na linha do que o Sr. Deputado António Filipe, com correção, disse, este debate nem sequer vem formulado como podendo ser um debate sobre corrupção neste contexto, ou sobre os dirigentes que são acusados de corrupção, ou sobre os dirigentes que, efetivamente, são condenados por corrupção, porque transformá-lo, como foi transformado, num debate sobre suspeição generalizada é tudo aquilo que não podemos fazer em democracia,… Aplausos do PS. … porque é um debate que assenta no princípio de que, à partida, por tocar numa área que, recordo, não é ilícita, é associativa, é o exercício de um direito fundamental, é o exercício da capacidade de intervir na vida de uma coletividade, que pode ser local ou nacional, que pode ser de futebol, mas pode ser de andebol ou de canoagem, pode ser de qualquer outra modalidade, como, aliás, os registos de interesses dos Srs. Deputados há décadas revelam. A quantidade de Sr.as e Srs. Deputados que participam, e participam corretamente, na vida das suas terras, dos seus clubes, porque é uma atividade cívica que louvamos e valorizamos noutros momentos, não podem agora ser rotulados com uma letra infamante na sua roupa com um F de futebol, como se isso significasse que, com isso, estão desqualificados da participação pública. Significa isto que não há um problema com os casos de corrupção? Não. Significa isto que a Assembleia da República não deve atender à forma como resolve o problema? Também não. Mas a pergunta é esta: este é o remédio eficiente? Não, também não é. Porquê? Porque o remédio eficiente é o que, em primeiro lugar, assenta na transparência. O Sr. Deputado André Silva referia a necessidade da transparência. Ora, o que nos traz hoje aqui a poder dizer e a poder verificar quantos foram, quantos são e quantos serão titulares de órgãos sociais é precisamente a transparência que é garantida através do registo de interesses de todas as Sr.as Deputadas e de todos os Srs. Deputados, onde está registado quais as funções associativas que exercem. Essa via garante essa transparência, que é a forma de os cidadãos controlarem e dizerem: «Não apreciei o facto de este Sr. Deputado estar associado àquela coletividade ou à direção daquela coletividade e disso retirarei as ilações que deverei retirar.» Eu, cidadão, livremente, não sou obrigado a fazer esse juízo por uma lei que se apresentaria como inadequada. Por outro lado, se continuarmos a olhar para a panóplia de instrumentos que temos ao nosso dispor, também a incompatibilidade não é a forma adequada. O conflito de interesses e a forma de o revelar é a forma que propiciamente serve para esta finalidade, como, aliás, ficou ilustrado, de forma bastante clara, pelo registo inicial do Sr. Deputado João Almeida. E mesmo que assim não fosse, se achássemos que o conflito de interesses não era necessário, há que verificar, então, se a figura do impedimento e se, por exemplo, a intervenção em debates parlamentares ou em estruturas de debate, em que, eventualmente, possa estar a ser discutida essa matéria, é pertinente ou não. E não é isso que se faz: independentemente de, hoje, um Deputado ser membro da comissão que tutela o desporto ou não, lá vai tudo com a mesma anatomização e a mesma marca infamante! E vir recordar exemplos das décadas de 80 e 90, quando, claramente, os titulares de cargos políticos perceberam que não podem ser presidentes da Federação Portuguesa de Futebol e titulares do mandato de Deputado ao mesmo tempo, porque a opinião pública a isso os levou, é também fazer um debate com duas décadas de atraso e não o debate que deveríamos estar aqui a fazer. Para sermos claros, o que está em causa é o debate da liberdade de associação, que envolve a participação nos órgãos sociais das coletividades. E sublinhe-se que seria a primeira vez na história da definição de incompatibilidades e impedimentos que se traçaria uma incompatibilidade não em relação a uma atividade profissional, mas em relação a uma atividade cívica e associativa. Esta é também uma linha vermelha que acho que a Assembleia da República não pode nem deve querer atravessar neste momento. Aplausos do PS. Há também uma incompreensão na intermitência das coisas que aqui aparecem neste diploma. São as competições profissionais, mas, aparentemente, já era o futebol todo e, aparentemente, não choca que um clube de bairro, pequeno, um clube das distritais possa, eventualmente, até ser alvo de processos ou de investigação por corrupção com a mesma acuidade que um clube de grande dimensão. Se formos ver, por exemplo, casos de corrupção associados a apostas desportivas, vemos que os clubes envolvidos nem sequer eram clubes profissionais. Se formos ver as transferências de jogadores de aqui para acolá, vemos que não são, muitas vezes, provenientes de clubes que não estariam sequer abrangidos por esta proposta, que até por isso se manifesta insuficiente. E o que seria da realidade de um clube em função da sua posição e da sua capacidade desportiva se subisse de divisão e, em função disso, ativássemos uma incompatibilidade ou incumprimento? Teríamos os Deputados, seguramente, a ver o final de uma tabela, a ver se desce ou se sobe para ver se estavam impedidos, incompatíveis ou não. Isto é burlesco demais e, regra geral, quando se torna burlesco torna-se manifesta e potencialmente uma matéria que não é a melhor solução nem o melhor caminho jurídico. Finalmente, temos tantas associações, tantas coletividades, tantas atividades e apenas esta foi anatomizada. Isto é um erro, porque tanto podemos encontrar conflito de interesses no futebol como, por exemplo, potencialmente, em dirigentes de associações de pais que estão na Comissão de Educação, em dirigentes de associações culturais que estão na Comissão de Cultura, em dirigentes de IPSS que estão na Comissão de Trabalho e Segurança Social e, citando o GatoFedorento, dirigentes do «Bigode» que possam estar na comissão da tutela respetiva. Portanto, Sr.as e Srs. Deputados, fundamentalmente, este é um debate sobre a dignidade do Parlamento, sobre como encaramos cada representante e como encaramos a sua relação com a sociedade civil. Por essa razão, não podemos acompanhar a proposta do PAN. Aplausos do PS."
print(len(long_txt))

7903


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')


tknz = sent_tokenize(long_text, language='portuguese')

ind = []
for i in range(len(tknz)):
    if tknz[i][-3:] == "Sr." or tknz[i][-4:] == "Srs.":
        ind.append(i)

for x in ind[::-1]:
    tknz[x] = tknz[x] + " " + tknz[x + 1]
    del(tknz[x])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tknz

['Deputados: O debate que nos é convocado aparenta ser um debate sobre o futebol e o impacto negativo do futebol, mas, na realidade, não é.',
 'Até vem à memória a frase do famoso treinador do Liverpool, Bill Shankly, que dizia: «O futebol não é uma questão de vida ou de morte, é muito mais importante do que isso».',
 'Na realidade, transpondo essa frase para este debate, o que deveríamos ter era um debate sobre incompatibilidades, impedimentos e conflito de interesses, que não é um debate de vida ou de morte, mas, na verdade, é muito mais importante do que isso, porque é um debate sobre a instituição, as regras que a regem e a forma como os mandatos são exercidos.',
 'Portanto, em primeiro lugar, este não é um debate sobre o que achamos ou deixamos de achar sobre o futebol, o mundo do futebol, os seus dirigentes, o que aí gostamos ou não gostamos.',
 'Portanto, não se trata de saber se vemos poesia num golo de Eusébio, se vemos brilhantismo numa tática dos Cinco Violinos ou se ficamos

In [ ]:
start = time.time()

encoded = list(map(lambda x: m2m100_tokenizer(x, return_tensors="pt"), tknz))
generated_tokens = list(map(lambda x: m2m100_model.generate(**x, forced_bos_token_id=m2m100_tokenizer.get_lang_id("en")), encoded))
translated = list(map(lambda x: m2m100_tokenizer.batch_decode(x, skip_special_tokens=True), generated_tokens))

end = time.time()

print("Tempo de Execução:", str(round(end - start, 4)) + "s")

Tempo de Execução: 455.0878s


In [ ]:
flat_translated = [sentence for embed_sentence in translated for sentence in embed_sentence]
long_translated = " ".join(flat_translated)

long_translated

'Deputies: The debate that is called to us seems to be a debate about football and the negative impact of football, but in reality it is not. Liverpool coach Bill Shankly said, “Football is not a matter of life or death, it’s much more important than that.” In fact, transposing this phrase to this debate, what we should have was a debate on incompatibilities, obstacles and conflict of interests, which is not a debate of life or death, but, in fact, is much more important than that, because it is a debate on the institution, the rules that govern it and how the mandates are exercised. So, first of all, this is not a debate about what we think or stop finding about football, the world of football, its leaders, what we like or don’t like there. Therefore, it is not about whether we see poetry in a goal of Eusebio, whether we see brilliantism in a tactic of the Five Violins, or whether we get without air when Vítor Baía makes a defense. This is not the point, and the debate should not be a

## Teste na Sumarização

### Teste com a frase do DataFrame

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

#Sumarização extrema
peg_xsum_model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum').to(device)
peg_xsum_tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')

#Sumarização Abstrativa
peg_model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large').to(device)
peg_tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')

#Sumarização Extrativa
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to(device)
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
df_translated = "Mr. President, Mr. and Mr. Deputies: At the end of the last Legislative Session, also as a political statement, I presented here the availability of the CDS to contribute, now in scrutiny, now in complement, to the urgent economic recovery of the country in the pandemic period. Here we are, Mr. Deputies, and it is with regret that I do not find that availability in the Government, to whom, of course, the primary responsibilities of this same mission belong. I read and listened carefully to the presentation of the said Strategic Vision for the Recovery Plan that we are discussing here today. And in all this process, I remember a old socialist rule: when the socialists don’t want to talk about what they haven’t done, they launch great debates about what they’ll never do. In addition to the already usual tendency to think in the form of a decade, I also noticed the need of Eng.o Costa Silva to present the more than 1000 contributions he received from Portuguese citizens with an ideological shape that belongs more to the PS than to these same citizens: “a historic defeat of the ultra-liberal ideas,” said Mr. Engineer. I found, Mr. and Mr. Deputies, truly extraordinary that a plan that proposes to think 10 years still comes embroidered in the rhetoric of the previous crisis, happily defeated, as should be, with the effort of all the Portuguese. Honestly, I don’t remember to hear, in this Chamber, a single voice to defend that the solution to win this new crisis goes by annuling the role of the State in the economy or in the Portuguese society, as, in other words, I don’t remember to hear in none of the previous legislatures. It is strange that, facing the scale of the challenges that we are facing today, Mrs. Members of the Government and other consultants insist on preferring the rhetoric to reality, the ideological removal to sustained contribution or the fire of view to concrete vision."

print("Número de caracteres:", len(df_translated))
print()

df_translated

Número de caracteres: 1917



'Mr. President, Mr. and Mr. Deputies: At the end of the last Legislative Session, also as a political statement, I presented here the availability of the CDS to contribute, now in scrutiny, now in complement, to the urgent economic recovery of the country in the pandemic period. Here we are, Mr. Deputies, and it is with regret that I do not find that availability in the Government, to whom, of course, the primary responsibilities of this same mission belong. I read and listened carefully to the presentation of the said Strategic Vision for the Recovery Plan that we are discussing here today. And in all this process, I remember a old socialist rule: when the socialists don’t want to talk about what they haven’t done, they launch great debates about what they’ll never do. In addition to the already usual tendency to think in the form of a decade, I also noticed the need of Eng.o Costa Silva to present the more than 1000 contributions he received from Portuguese citizens with an ideologic

In [ ]:
# Sumarização Extrema

batch = peg_xsum_tokenizer(df_translated, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = peg_xsum_model.generate(**batch)
xsum_output = peg_xsum_tokenizer.batch_decode(translated, skip_special_tokens=True)

In [ ]:
xsum_output[0]

'Here is the full text of the speech by CDS leader Pedro Passos Coelho in the Portuguese Parliament on Wednesday.'

Apesar de bem sumarizado e "direto" o xsum tem uma alucinação quanto a data "Wednesday", uma vez que em nenhum momento do texto de entrada é mencionado essa data a não ser se isso for uma interpretação de "today"

In [ ]:
# Sumarização Abstrativa

peg_model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large').to(device)
peg_tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')

#truncation=True, padding='longest', max_length=1024,

inputs = peg_tokenizer(df_translated, return_tensors='pt').to(device)
summary_ids = peg_model.generate(inputs['input_ids'])
peg_output = [peg_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

In [ ]:
peg_output[0]

'In addition to the already usual tendency to think in the form of a decade, I also noticed the need of Eng.o Costa Silva to present the more than 1000 contributions he received from Portuguese citizens with an ideological shape that belongs more to the PS than to these same citizens: “a historic defeat of the ultra-liberal ideas,” said Mr. Deputies, truly extraordinary that a plan that proposes to think 10 years still comes embroidered in the rhetoric of the previous crisis, happily defeated, as should be, with the effort of all the Portuguese.'

Bem melhor que o XSum, mas ele apresenta uma pequena imprecisão de referência com o said "Mr. Deputies" que deveria ser "Mr. Engineer" se referindo ao Costa Silva

In [ ]:
# Sumarização Extrativa

inputs = bart_tokenizer(df_translated, truncation=True, padding='longest', max_length=1024, return_tensors='pt').to(device)
summary_ids = bart_model.generate(inputs['input_ids'])
bart_output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

In [ ]:
bart_output[0]

'Mr. President, Mr. and Mr. Deputies: At the end of the last Legislative Session, also as a political statement, I presented here the availability of the CDS to contribute, now in scrutiny, to the urgent economic recovery of the country in the pandemic period. I read and listened carefully to the presentation of the said Strategic Vision for the Recovery Plan that we are discussing here today.'

### Teste com Frase longa

In [ ]:
long_translated = "Deputies: The debate that is called to us seems to be a debate about football and the negative impact of football, but in reality it is not. Liverpool coach Bill Shankly said, “Football is not a matter of life or death, it’s much more important than that.” In fact, transposing this phrase to this debate, what we should have was a debate on incompatibilities, obstacles and conflict of interests, which is not a debate of life or death, but, in fact, is much more important than that, because it is a debate on the institution, the rules that govern it and how the mandates are exercised. So, first of all, this is not a debate about what we think or stop finding about football, the world of football, its leaders, what we like or don’t like there. Therefore, it is not about whether we see poetry in a goal of Eusebio, whether we see brilliantism in a tactic of the Five Violins, or whether we get without air when Vítor Baía makes a defense. This is not the point, and the debate should not be about football. If we turn it into a football debate, we remove the debate from where it should be. Deputy André Silva said one thing with which it is possible to agree — that today there is a debate poisoned in the public opinion, and there are television shows that poison the way citizens relate to each other, that instigate hatred, instigate differences among others and that is politically exploited by some, and that, therefore, must be denounced. But from claim to prohibition goes a quite different way. Deputy António Filipe, with correction, said, this debate does not even come formulated as being a debate on corruption in this context, or on the leaders who are accused of corruption, or on the leaders who are effectively condemned for corruption, because transforming it, as it has been transformed, into a debate on widespread suspicion is all that we cannot do in democracy. Members have revealed it for decades. Deputies who participate, and participate properly, in the life of their lands, their clubs, because it is a civic activity that we praise and value in other moments, cannot now be labelled with an infamous letter in their clothes with a football F, as if that meant that, with that, they are disqualified from public participation. Does this mean that there is no problem with corruption cases? Not to. Does this mean that the Assembly of the Republic should not address the way it solves the problem? Also not. But the question is this: is this the effective remedy? No, it is also not. and why? Because the effective remedy is what, first of all, is based on transparency. Deputy André Silva referred to the need for transparency. Members, where it is registered which associative functions they perform. Deputy be associated with that collectivity or the direction of that collectivity and from it I will withdraw the additions that I will have to withdraw.”I, citizen, freely, am not obliged to make that judgment by a law that would be presented as inadequate. On the other hand, if we continue to look at the panopoly of instruments we have at our disposal, also incompatibility is not the right way. Deputy Joao Almeida. And even if so was not, if we found that the conflict of interest was not necessary, then it is necessary to verify whether the figure of the impediment and whether, for example, the intervention in parliamentary debates or in debate structures, in which, eventually, may be being discussed that matter, is relevant or not. And this is not what is done: regardless of whether today a Deputy is a member of the committee that protects the sport or not, there goes everything with the same anatomization and the same infamous mark! And come to remember examples of the 80s and 90s, when, clearly, the holders of political positions realized that they cannot be presidents of the Portuguese Football Federation and holders of the mandate of Deputy at the same time, because the public opinion has led them to that, it is also to do a debate with two decades of delay and not the debate that we should be here to do. To be clear, what is in question is the debate of freedom of association, which involves participation in the social organs of the collectivities. This would be the first time in the history of the definition of incompatibilities and obstacles that an incompatibility would be traced not in relation to a professional activity, but in relation to a civil and associative activity. This is also a red line that I think the Assembly of the Republic cannot and should not want to cross at this moment. Congratulations from PS. There is also an incomprehension in the intermittence of the things that appear here in this diploma. They are the professional competitions, but apparently it was already the whole football and apparently it is not shocking that a neighborhood club, small, a district club may eventually even be targeted by trials or investigations for corruption with the same sharpness as a large club. If we see, for example, cases of corruption associated with sports betting, we see that the clubs involved were not even professional clubs. If we go to see the transfer of players from here to here, we see that they are not often from clubs that would not even be covered by this proposal, which even therefore shows insufficient. And what would be the reality of a club depending on its position and its sports capacity if it came up of division and, as a result, we would activate an incompatibility or non-compliance? We would have the Deputies,, to see the end of a table, to see if it descends or rises to see if they were prevented, incompatible or not. This is too burlesque and, as a rule, when it becomes burlesque it becomes manifest and potentially a matter that is not the best solution nor the best legal path. Finally, we have so many associations, so many collectivities, so many activities and only this has been anatomized. This is a mistake, because we can both find conflict of interests in football as, for example, potentially, in the leaders of parental associations who are in the Education Commission, in the leaders of cultural associations who are in the Culture Commission, in the leaders of IPSS who are in the Labour and Social Security Commission and, citing the CatFedorento, the leaders of the “Bigode” who may be in the committee of their custody. Deputies, fundamentally, this is a debate on the dignity of the Parliament, on how we look to each representative and how we look to their relationship with civil society. For this reason, we cannot follow the proposal of the PAN. Congratulations from PS."
print("Número de caracteres:", len(long_translated))
print()

long_translated

Número de caracteres: 6622



'Deputies: The debate that is called to us seems to be a debate about football and the negative impact of football, but in reality it is not. Liverpool coach Bill Shankly said, “Football is not a matter of life or death, it’s much more important than that.” In fact, transposing this phrase to this debate, what we should have was a debate on incompatibilities, obstacles and conflict of interests, which is not a debate of life or death, but, in fact, is much more important than that, because it is a debate on the institution, the rules that govern it and how the mandates are exercised. So, first of all, this is not a debate about what we think or stop finding about football, the world of football, its leaders, what we like or don’t like there. Therefore, it is not about whether we see poetry in a goal of Eusebio, whether we see brilliantism in a tactic of the Five Violins, or whether we get without air when Vítor Baía makes a defense. This is not the point, and the debate should not be a

In [ ]:
# Sumarização Extrema

batch = peg_xsum_tokenizer(long_translated, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = peg_xsum_model.generate(**batch)
xsum_output = peg_xsum_tokenizer.batch_decode(translated, skip_special_tokens=True)

In [ ]:
xsum_output[0]

'The following is the text of a debate in the Spanish parliament on corruption in football.'

In [ ]:
# Sumarização Abstrativa

inputs = peg_tokenizer(long_translated, return_tensors='pt').to(device)
summary_ids = peg_model.generate(inputs['input_ids']).to(device)
peg_output = [peg_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

Token indices sequence length is longer than the specified maximum sequence length for this model (1362 > 1024). Running this sequence through the model will result in indexing errors


IndexError: ignored

In [ ]:
peg_output[0]

'Liverpool coach Bill Shankly said, “Football is not a matter of life or death, it’s much more important than that.” In fact, transposing this phrase to this debate, what we should have was a debate on incompatibilities, obstacles and conflict of interests, which is not a debate of life or death, but, in fact, is much more important than that, because it is a debate on the institution, the rules that govern it and how the mandates are exercised. Deputy André Silva said one thing with which it is possible to agree — that today there is a debate in the public opinion, and there are television shows that poison the way citizens relate to each other, that instigate hatred, instigate differences among others and that is politically exploited by some, and that, therefore, must be denounced. Deputy Antnio Filipe, with correction, said, this debate does not even come formulated as being a debate on corruption in this context, or on the leaders who are accused of corruption, or on the leaders w

In [ ]:
peg_output[0]

In [ ]:
# Sumarização Extrativa

inputs = bart_tokenizer(long_translated, truncation=True, padding='longest', max_length=1024, return_tensors='pt').to(device)
summary_ids = bart_model.generate(inputs['input_ids'])
bart_output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

In [ ]:
bart_output[0]

'Deputies: The debate that is called to us seems to be a debate about football and the negative impact of football, but in reality it is not. It is a debate on the institution, the rules that govern it and how the mandates are exercised. Deputies who participate, and participate properly, in the life of their lands, their clubs cannot now be labelled with an infamous letter in their clothes with a football F.'

In [ ]:
long_translated

'Deputies: The debate that is called to us seems to be a debate about football and the negative impact of football, but in reality it is not. Liverpool coach Bill Shankly said, “Football is not a matter of life or death, it’s much more important than that.” In fact, transposing this phrase to this debate, what we should have was a debate on incompatibilities, obstacles and conflict of interests, which is not a debate of life or death, but, in fact, is much more important than that, because it is a debate on the institution, the rules that govern it and how the mandates are exercised. So, first of all, this is not a debate about what we think or stop finding about football, the world of football, its leaders, what we like or don’t like there. Therefore, it is not about whether we see poetry in a goal of Eusebio, whether we see brilliantism in a tactic of the Five Violins, or whether we get without air when Vítor Baía makes a defense. This is not the point, and the debate should not be a

In [ ]:
print(ablublueble)

NameError: ignored

In [ ]:
long_translated

'Deputies: The debate that is called to us seems to be a debate about football and the negative impact of football, but in reality it is not. Liverpool coach Bill Shankly said, “Football is not a matter of life or death, it’s much more important than that.” In fact, transposing this phrase to this debate, what we should have was a debate on incompatibilities, obstacles and conflict of interests, which is not a debate of life or death, but, in fact, is much more important than that, because it is a debate on the institution, the rules that govern it and how the mandates are exercised. So, first of all, this is not a debate about what we think or stop finding about football, the world of football, its leaders, what we like or don’t like there. Therefore, it is not about whether we see poetry in a goal of Eusebio, whether we see brilliantism in a tactic of the Five Violins, or whether we get without air when Vítor Baía makes a defense. This is not the point, and the debate should not be a

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')


In [ ]:
print(len(long_translated))

6622


In [ ]:
6580/3

2193.3333333333335

In [ ]:
count = 0
sentences = sent_tokenize(long_translated)

for i in range(len(sentences)):
    count += len(sentences[i])
    if count > 2190:
        count = 0
        print(i)

10
29


In [ ]:
sentences

['Deputies: The debate that is called to us seems to be a debate about football and the negative impact of football, but in reality it is not.',
 'Liverpool coach Bill Shankly said, “Football is not a matter of life or death, it’s much more important than that.” In fact, transposing this phrase to this debate, what we should have was a debate on incompatibilities, obstacles and conflict of interests, which is not a debate of life or death, but, in fact, is much more important than that, because it is a debate on the institution, the rules that govern it and how the mandates are exercised.',
 'So, first of all, this is not a debate about what we think or stop finding about football, the world of football, its leaders, what we like or don’t like there.',
 'Therefore, it is not about whether we see poetry in a goal of Eusebio, whether we see brilliantism in a tactic of the Five Violins, or whether we get without air when Vítor Baía makes a defense.',
 'This is not the point, and the debat

In [ ]:
first = " ".join(sentences[:11])
second = " ".join(sentences[11:30])
third = " ".join(sentences[30:-1])

In [ ]:
input_1 = peg_tokenizer(first, truncation=True, padding='longest', max_length=1024, return_tensors='pt').to(device)
input_2 = peg_tokenizer(second, truncation=True, padding='longest', max_length=1024, return_tensors='pt').to(device)
input_3 = peg_tokenizer(third, truncation=True, padding='longest', max_length=1024, return_tensors='pt').to(device)

In [ ]:
summary_ids_1 = peg_model.generate(input_1['input_ids'])
summary_ids_2 = peg_model.generate(input_2['input_ids'])
summary_ids_3 = peg_model.generate(input_3['input_ids'])

In [ ]:
bart_output_1 = [peg_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_1]
bart_output_2 = [peg_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_2]
bart_output_3 = [peg_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_3]

In [ ]:
bart_output_1[0]

'Liverpool coach Bill Shankly said, “Football is not a matter of life or death, it’s much more important than that.” In fact, transposing this phrase to this debate, what we should have was a debate on incompatibilities, obstacles and conflict of interests, which is not a debate of life or death, but, in fact, is much more important than that, because it is a debate on the institution, the rules that govern it and how the mandates are exercised. Deputy Antnio Filipe, with correction, said, this debate does not even come formulated as being a debate on corruption in this context, or on the leaders who are accused of corruption, or on the leaders who are effectively condemned for corruption, because transforming it, as it has been transformed, into a debate on widespread suspicion is all that we cannot do in democracy.'

In [ ]:
bart_output_2[0]

'Does this mean that the Assembly of the Republic should not address the way it solves the problem? And come to remember examples of the 80s and 90s, when, clearly, the holders of political positions realized that they cannot be presidents of the Portuguese Football Federation and holders of the mandate of Deputy at the same time, because the public opinion has led them to that, it is also to do a debate with two decades of delay and not the debate that we should be here to do. This would be the first time in the history of the definition of incompatibilities and obstacles that an incompatibility would be traced not in relation to a professional activity, but in relation to a civil and associative activity.'

In [ ]:
bart_output_3[0]

'If we see, for example, cases of corruption associated with sports betting, we see that the clubs involved were not even professional clubs.'

In [ ]:
# Sumarização Abstrativa
txt = bart_output_1[0] + " " + bart_output_2[0] + " " + bart_output_3[0]
inputs = peg_tokenizer(txt, return_tensors='pt').to(device)
summary_ids = peg_model.generate(inputs['input_ids']).to(device)
peg_output = [peg_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

In [ ]:
peg_output[0]

'Deputy Antnio Filipe, with correction, said, this debate does not even come formulated as being a debate on corruption in this context, or on the leaders who are accused of corruption, or on the leaders who are effectively condemned for corruption, because transforming it, as it has been transformed, into a debate on widespread suspicion is all that we cannot do in democracy.'

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

m2m100_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="pt", tgt_lang="en")
m2m100_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

m2m100_tokenizer.src_lang = "en"

txt = 'Deputy Antnio Filipe, with correction, said, this debate does not even come formulated as being a debate on corruption in this context, or on the leaders who are accused of corruption, or on the leaders who are effectively condemned for corruption, because transforming it, as it has been transformed, into a debate on widespread suspicion is all that we cannot do in democracy.'

encoded = m2m100_tokenizer(txt, return_tensors="pt")
generated_tokens = m2m100_model.generate(**encoded, forced_bos_token_id=m2m100_tokenizer.get_lang_id("pt"))
df_translated = m2m100_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [ ]:
df_translated[0]

'O deputado Antnio Filipe, com correção, disse que este debate nem se formulou como um debate sobre corrupção neste contexto, ou sobre os líderes que são acusados de corrupção, ou sobre os líderes que são efetivamente condenados por corrupção, porque transformá-lo, como foi transformado, em um debate sobre suspeitas amplas é tudo que não podemos fazer na democracia.'

In [ ]:
inputs = bart_tokenizer(long_translated, truncation=True, padding='longest', max_length=1024, return_tensors='pt').to(device)
summary_ids = bart_model.generate(inputs['input_ids'])
bart_output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

['Does this mean that there is no problem with corruption cases?',
 'Not to.',
 'Does this mean that the Assembly of the Republic should not address the way it solves the problem?',
 'Also not.',
 'But the question is this: is this the effective remedy?',
 'No, it is also not.',
 'and why?',
 'Because the effective remedy is what, first of all, is based on transparency.',
 'Deputy André Silva referred to the need for transparency.',
 'Members, where it is registered which associative functions they perform.',
 'Deputy be associated with that collectivity or the direction of that collectivity and from it I will withdraw the additions that I will have to withdraw.”I, citizen, freely, am not obliged to make that judgment by a law that would be presented as inadequate.',
 'On the other hand, if we continue to look at the panopoly of instruments we have at our disposal, also incompatibility is not the right way.',
 'Deputy Joao Almeida.',
 'And even if so was not, if we found that the confl

## Teste na 2a Tradução

## Teste completo

In [ ]:
import pandas as pd
df = pd.read_excel("output (3) (1).xlsx")
df[df["Text"].str.contains("corrup")].reset_index().iloc[74]

In [ ]:
df[df["Text"].str.contains("corrup")]

In [ ]:
df.iloc[9596:9599]

In [ ]:
text = df[df["Text"].str.contains("corrup")].reset_index().iloc[74]["Text"]

In [ ]:
#Maior texto que menciona corrupção
# https://debates.parlamento.pt/catalogo/r3/dar/01/14/02/041/2021-01-20/41
#text = df[df["Text"].str.contains("corrup")].reset_index().iloc[74]["Text"]
import pandas as pd
df = pd.read_csv("/content/dialog_lines_database_initiatives.csv")
text = df.iloc[9596]["Text"] + " " + df.iloc[9598]["Text"]
text

In [ ]:
len(text)

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize

tknz = sent_tokenize(text, language='portuguese')

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize

tknz = sent_tokenize(text, language='portuguese')

for i in range(len(tknz)):
    if tknz[i][-3:] == "Sr." or tknz[i][-4:] == "Srs.":
        print(i)
    
ind = [
0,
8,
11,
13,
15,
25,
27,
29,
32,
50,
]

for x in ind[::-1]:
    tknz[x] = tknz[x] + " " + tknz[x + 1]
    del(tknz[x])

In [ ]:
ind = [
0,
8,
11,
13,
15,
25,
27,
29,
32,
50,
]

for x in ind[::-1]:
    tknz[x] = tknz[x] + " " + tknz[x + 1]
    del(tknz[x])

In [ ]:
tknz[23] = "Por outro lado, se continuarmos a olhar para a panóplia de instrumentos que temos ao nosso dispor, também a incompatibilidade não é a forma adequada."

In [ ]:
tknz

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import time

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
tokenizer.src_lang = "pt"

texts = pd.DataFrame(tknz)
texts.columns = ["Text"]

start = time.time()

texts["Encoded"] = texts["Text"].apply(lambda x: tokenizer(x, return_tensors="pt"))
texts["generated_tokens"] = texts["Encoded"].apply(lambda x: model.generate(**x, forced_bos_token_id=tokenizer.get_lang_id("en")))
texts["translated"] = texts["generated_tokens"].apply(lambda x: tokenizer.batch_decode(x, skip_special_tokens=True))

end = time.time()

print("Tempo de Execução:", str(round(end - start, 4)) + "s")

In [ ]:
texts["translated"]

In [ ]:
ARTICLE_TO_SUMMARIZE = ""

for txt in texts["translated"].tolist():
    ARTICLE_TO_SUMMARIZE += " " + txt[0]

ARTICLE_TO_SUMMARIZE

In [ ]:
ARTICLE_TO_SUMMARIZE_1 = "Deputies: The debate that is called to us seems to be a debate about football and the negative impact of football, but in reality it is not. Liverpool coach Bill Shankly said, “Football is not a matter of life or death, it’s much more important than that.” In fact, transposing this phrase to this debate, what we should have was a debate on incompatibilities, obstacles and conflict of interests, which is not a debate of life or death, but, in fact, is much more important than that, because it is a debate on the institution, the rules that govern it and how the mandates are exercised. So, first of all, this is not a debate about what we think or stop finding about football, the world of football, its leaders, what we like or don’t like there. Therefore, it is not about whether we see poetry in a goal of Eusebio, whether we see brilliantism in a tactic of the Five Violins, or whether we get without air when Vítor Baía makes a defense. This is not the point, and the debate should not be about football. If we turn it into a football debate, we remove the debate from where it should be. Deputy André Silva said one thing with which it is possible to agree — that today there is a debate poisoned in the public opinion, and there are television shows that poison the way citizens relate to each other, that instigate hatred, instigate differences among others and that is politically exploited by some, and that, therefore, must be denounced. But from claim to prohibition goes a quite different way. Deputy António Filipe, with correction, said, this debate does not even come formulated as being a debate on corruption in this context, or on the leaders who are accused of corruption, or on the leaders who are effectively condemned for corruption, because transforming it, as it has been transformed, into a debate on widespread suspicion is all that we cannot do in democracy. Members have revealed it for decades. Deputies who participate, and participate properly, in the life of their lands, their clubs, because it is a civic activity that we praise and value in other moments, cannot now be labelled with an infamous letter in their clothes with a football F, as if that meant that, with that, they are disqualified from public participation. Does this mean that there is no problem with corruption cases? Not to. Does this mean that the Assembly of the Republic should not address the way it solves the problem? Also not. But the question is this: is this the effective remedy? No, it is also not. and why? Because the effective remedy is what, first of all, is based on transparency. Deputy André Silva referred to the need for transparency. Members, where it is registered which associative functions they perform. Deputy be associated with that collectivity or the direction of that collectivity and from it I will withdraw the additions that I will have to withdraw.”I, citizen, freely, am not obliged to make that judgment by a law that would be presented as inadequate."
ARTICLE_TO_SUMMARIZE_2 = "On the other hand, if we continue to look at the panopoly of instruments we have at our disposal, also incompatibility is not the right way. Deputy Joao Almeida. And even if so was not, if we found that the conflict of interest was not necessary, then it is necessary to verify whether the figure of the impediment and whether, for example, the intervention in parliamentary debates or in debate structures, in which, eventually, may be being discussed that matter, is relevant or not. And this is not what is done: regardless of whether today a Deputy is a member of the committee that protects the sport or not, there goes everything with the same anatomization and the same infamous mark! And come to remember examples of the 80s and 90s, when, clearly, the holders of political positions realized that they cannot be presidents of the Portuguese Football Federation and holders of the mandate of Deputy at the same time, because the public opinion has led them to that, it is also to do a debate with two decades of delay and not the debate that we should be here to do. To be clear, what is in question is the debate of freedom of association, which involves participation in the social organs of the collectivities. This would be the first time in the history of the definition of incompatibilities and obstacles that an incompatibility would be traced not in relation to a professional activity, but in relation to a civil and associative activity. This is also a red line that I think the Assembly of the Republic cannot and should not want to cross at this moment. Congratulations from PS. There is also an incomprehension in the intermittence of the things that appear here in this diploma. They are the professional competitions, but apparently it was already the whole football and apparently it is not shocking that a neighborhood club, small, a district club may eventually even be targeted by trials or investigations for corruption with the same sharpness as a large club. If we see, for example, cases of corruption associated with sports betting, we see that the clubs involved were not even professional clubs. If we go to see the transfer of players from here to here, we see that they are not often from clubs that would not even be covered by this proposal, which even therefore shows insufficient. And what would be the reality of a club depending on its position and its sports capacity if it came up of division and, as a result, we would activate an incompatibility or non-compliance? We would have the Deputies,, to see the end of a table, to see if it descends or rises to see if they were prevented, incompatible or not. This is too burlesque and, as a rule, when it becomes burlesque it becomes manifest and potentially a matter that is not the best solution nor the best legal path. Finally, we have so many associations, so many collectivities, so many activities and only this has been anatomized. This is a mistake, because we can both find conflict of interests in football as, for example, potentially, in the leaders of parental associations who are in the Education Commission, in the leaders of cultural associations who are in the Culture Commission, in the leaders of IPSS who are in the Labour and Social Security Commission and, citing the CatFedorento, the leaders of the “Bigode” who may be in the committee of their custody. Deputies, fundamentally, this is a debate on the dignity of the Parliament, on how we look to each representative and how we look to their relationship with civil society. For this reason, we cannot follow the proposal of the PAN. Congratulations from PS."

In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large')
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')

inputs_1 = tokenizer([ARTICLE_TO_SUMMARIZE_1], return_tensors='pt')
inputs_2 = tokenizer([ARTICLE_TO_SUMMARIZE_2], return_tensors='pt')
# Generate Summary
summary_ids_1 = model.generate(inputs_1['input_ids'], )
summary_ids_2 = model.generate(inputs_2['input_ids'], )

summary_peg_1 = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_1]
summary_peg_2 = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids_2]

In [ ]:
summary_peg_1[0] + ' ' + summary_peg_2[0]

In [ ]:
print(len(ARTICLE_TO_SUMMARIZE_1 + " " + ARTICLE_TO_SUMMARIZE_2))
print(len(summary_peg_1[0] + ' ' + summary_peg_2[0]))

ARTICLE_TO_SUMMARIZE = "Liverpool coach Bill Shankly said, “Football is not a matter of life or death, it’s much more important than that.” In fact, transposing this phrase to this debate, what we should have was a debate on incompatibilities, obstacles and conflict of interests, which is not a debate of life or death, but, in fact, is much more important than that, because it is a debate on the institution, the rules that govern it and how the mandates are exercised. Deputy André Silva said one thing with which it is possible to agree — that today there is a debate poisoned in the public opinion, and there are television shows that poison the way citizens relate to each other, that instigate hatred, instigate differences among others and that is politically exploited by some, and that, therefore, must be denounced. Deputy António Filipe, with correction, said, this debate does not even come formulated as being a debate on corruption in this context, or on the leaders who are accused of corruption, or on the leaders who are effectively condemned for corruption, because transforming it, as it has been transformed, into a debate on widespread suspicion is all that we cannot do in democracy. And even if so was not, if we found that the conflict of interest was not necessary, then it is necessary to verify whether the figure of the impediment and whether, for example, the intervention in parliamentary debates or in debate structures, in which, eventually, may be being discussed that matter, is relevant or not. This would be the first time in the history of the definition of incompatibilities and obstacles that an incompatibility would be traced not in relation to a professional activity, but in relation to a civil and associative activity."

In [ ]:
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large')
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')

inputs = tokenizer([ARTICLE_TO_SUMMARIZE], return_tensors='pt')
# Generate Summary
summary_ids = model.generate(inputs['input_ids'], )

summary_peg = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

In [ ]:
summary_peg[0]

In [ ]:
sentences = [
             
]

In [ ]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
tokenizer.src_lang = "en"

encoded = list(map(lambda x: tokenizer(x, return_tensors="pt"), sentences))
generated_tokens = list(map(lambda x: model.generate(**x, forced_bos_token_id=tokenizer.get_lang_id("pt")), encoded))
translated = list(map(lambda x: tokenizer.batch_decode(x, skip_special_tokens=True), generated_tokens))

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
import torch


model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

#ARTICLE_TO_SUMMARIZE = "Mr. President, Mr. and Mr. Deputies: At the end of the last Legislative Session, also as a political statement, I presented here the availability of the CDS to contribute, now in scrutiny, now in complement, to the urgent economic recovery of the country in the pandemic period. Here we are, Mr. Deputies, and it is with regret that I do not find that availability in the Government, to whom, of course, the primary responsibilities of this same mission belong. I read and listened carefully to the presentation of the said Strategic Vision for the Recovery Plan that we are discussing here today. And in all this process, I remember a old socialist rule: when the socialists don’t want to talk about what they haven’t done, they launch great debates about what they’ll never do. In addition to the already usual tendency to think in the form of a decade, I also noticed the need of Eng.o Costa Silva to present the more than 1000 contributions he received from Portuguese citizens with an ideological shape that belongs more to the PS than to these same citizens: “a historic defeat of the ultra-liberal ideas,” said Mr. Engineer. I found, Mr. and Mr. Deputies, truly extraordinary that a plan that proposes to think 10 years still comes embroidered in the rhetoric of the previous crisis, happily defeated, as should be, with the effort of all the Portuguese. Honestly, I don’t remember to hear, in this Chamber, a single voice to defend that the solution to win this new crisis goes by annuling the role of the State in the economy or in the Portuguese society, as, in other words, I don’t remember to hear in none of the previous legislatures. It is strange that, facing the scale of the challenges that we are facing today, Mrs. Members of the Government and other consultants insist on preferring the rhetoric to reality, the ideological removal to sustained contribution or the fire of view to concrete vision."

inputs = tokenizer([ARTICLE_TO_SUMMARIZE], return_tensors='pt')

# Generate Summary
summary_ids = model.generate(inputs['input_ids'],) #num_beams=4, max_length=5, early_stopping=True)
summary_bart = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

In [ ]:
summary_bart[0]

In [ ]:
sentences = [
"Deputy André Silva said today there is a debate poisoned in the public opinion.",
"There are television shows that instigate hatred, instigate differences among others, and that, therefore, must be denounced.",
"Deputy António Filipe, with correction, said, this debate does not even come formulated as being a debate on corruption in this context.",
]

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
tokenizer.src_lang = "en"

encoded = list(map(lambda x: tokenizer(x, return_tensors="pt"), sentences))
generated_tokens = list(map(lambda x: model.generate(**x, forced_bos_token_id=tokenizer.get_lang_id("pt")), encoded))
translated = list(map(lambda x: tokenizer.batch_decode(x, skip_special_tokens=True), generated_tokens))

In [ ]:
txt = 'O deputado André Silva disse hoje que há um debate envenenado na opinião pública. Há programas de televisão que estimulam o ódio, estimulam diferenças entre outros, e que, portanto, devem ser denunciados. O deputado António Filipe, com correção, disse que este debate nem se formulou como um debate sobre corrupção neste contexto.'
txt

In [ ]:
sentences = [
             "Liverpool coach Bill Shankly said, “Football is not a matter of life or death, it’s much more important than that.”",
            "In fact, transposing this phrase to this debate, what we should have was a debate on incompatibilities, obstacles and conflict of interests, which is not a debate of life or death, but, in fact, is much more important than that, because it is a debate on the institution, the rules that govern it and how the mandates are exercised."
]

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")
tokenizer.src_lang = "en"

encoded = list(map(lambda x: tokenizer(x, return_tensors="pt"), sentences))
generated_tokens = list(map(lambda x: model.generate(**x, forced_bos_token_id=tokenizer.get_lang_id("pt")), encoded))
translated = list(map(lambda x: tokenizer.batch_decode(x, skip_special_tokens=True), generated_tokens))

In [ ]:
txt = 'O treinador de Liverpool, Bill Shankly, disse que “o futebol não é uma questão de vida ou morte, é muito mais importante do que isso”. De fato, transferindo esta frase para este debate, o que devemos ter foi um debate sobre incompatibilidades, obstáculos e conflito de interesses, que não é um debate de vida ou morte, mas, de fato, é muito mais importante do que isso, porque é um debate sobre a instituição, as regras que a governam e como os mandatos são exercidos.'
txt

'O treinador de Liverpool, Bill Shankly, disse que “o futebol não é uma questão de vida ou morte, é muito mais importante do que isso”. De fato, transferindo esta frase para este debate, o que devemos ter foi um debate sobre incompatibilidades, obstáculos e conflito de interesses, que não é um debate de vida ou morte, mas, de fato, é muito mais importante do que isso, porque é um debate sobre a instituição, as regras que a governam e como os mandatos são exercidos.'

In [ ]:
text = df.iloc[9596]["Text"] + " " + df.iloc[9598]["Text"]
print("Número de caracteres original:...", len(text[0]))
print("Número de caracteres pegasus:....", len(summary_peg[0]))
print("Número de caracteres BART:.......", len(summary_bart[0]))

In [ ]:
#Exemplo de declaração política que cita corrupção
discussion = df[(df["Transcript"] == "DAR-001") & (df["Subject"] == "Declarações políticas 6")]
discussion

In [ ]:
#Exemplo de projeto que cita a corrupção: Petição n.º 13/XIV/1.ª (algoritmo não registra como Subject)
discussion = df.iloc[309:356]

## BERTopic

In [ ]:
!pip install bertopic

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(language="portuguese",
                       calculate_probabilities=True,
                       verbose=True,
                       n_gram_range=(1, 1),
                       top_n_words=7,
                       nr_topics="auto",
                       )
text_list = df['Text'].tolist()
topics, probabilities = topic_model.fit_transform(text_list)

In [ ]:
all_topics = topic_model.get_topics()
all_topics
melhores = [4, 8, 9, 11, 17, 33, 36, 46, 48, 63, 95, 115, 118, 120]

In [ ]:
melhores = [4, 8, 9, 11, #17, 33, 46, 115,
            36, 48, 63, 95, 118, 120]
topic_model.visualize_barchart(topics=melhores)

In [ ]:
topic_model.visualize_barchart(topics=melhores[:9])

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import torch
import time 

df = pd.read_excel("output (3) (1).xlsx")

# No DF inteiro

Uma rodada de PEGASUS no DF inteiro

In [ ]:
!pip install SentencePiece

In [ ]:
!pip install transformers

In [ ]:
import tqdm
from tqdm.notebook import tqdm_notebook
from tqdm import trange
import pandas as pd

tqdm_notebook.pandas()
df = pd.DataFrame()

In [ ]:
df["Falas"] = [
    "Sr. Presidente, Srs. Membros do Governo, Sr.as e Srs. Deputados: Depois de o PSD e de o CDS-PP terem aprovado a proposta de lei do Governo Passos Coelho/Paulo Portas, que viria dar origem à Lei n.º 22/2012 e materializada pela Lei n.º 11-A/2013, a nossa democracia ficou substancialmente mais pobre. Na verdade, apesar da pomposa designação «Reorganização administrativa do território das freguesias», esta lei mais não veio fazer do que impor a extinção de freguesias, que foi, aliás, o único objetivo pretendido, ainda que não assumido pelo Governo PSD/CDS, com esta dita «reorganização administrativa». A contestação ao processo de extinção de freguesias foi, e é, pública e notória e estende-se a toda a gente e aos mais variados setores, desde logo às autarquias. pretende usar da palavra antes dela fazer a intervenção de encerramento. Se a resposta for negativa, não pode haver perguntas. Sr. Presidente, Srs. Membros do Governo, Sr.as e Srs. Deputados: Depois de o PSD e de o CDS-PP terem aprovado a proposta de lei do Governo Passos Coelho/Paulo Portas, que viria dar origem à Lei n.º 22/2012 e materializada pela Lei n.º 11-A/2013, a nossa democracia ficou substancialmente mais pobre. Na verdade, apesar da pomposa designação «Reorganização administrativa do território das freguesias», esta lei mais não veio fazer do que impor a extinção de freguesias, que foi, aliás, o único objetivo pretendido, ainda que não assumido pelo Governo PSD/CDS, com esta dita «reorganização administrativa». A contestação ao processo de extinção de freguesias foi, e é, pública e notória e estende-se a toda a gente e aos mais variados setores, desde logo às autarquias. Foram centenas e centenas os e-mails de cidadãos, os ofícios de assembleias municipais e de assembleias de freguesia de todo o País que chegaram ao Grupo Parlamentar «Os Verdes» e, presumo, também aos restantes grupos parlamentares, dando conta da sua oposição à extinção de freguesias. Por várias vezes, milhares de autarcas de todo o País organizaram vigílias, manifestações e protestos contra este processo. A ANAFRE e a Associação Nacional de Municípios opuseram-se, de forma determinada, aos propósitos desta dita «reorganização administrativa». Os motivos desta generalizada contestação residem sobretudo, por um lado, no facto de o Governo de então não ter atribuído qualquer relevância à «pronúncia» das próprias autarquias sobre o assunto e, por outro lado, porque o que esteve na origem deste processo foram elementos estranhos à própria organização administrativa. De facto, o que motivou esta decisão não foram os interesses das populações, e cedo os portugueses e os autarcas perceberam que este processo nada tinha a ver com a preocupação em melhorar a qualidade de vida das pessoas. Esta reforma foi desenhada a pensar em tudo menos nas pessoas, nos seus problemas e nas suas preocupações. O propósito deste processo foi, exclusivamente, a redução do número de autarquias, mesmo que isso implicasse, como implicou e como continua a implicar, sacrifícios e mais dificuldades na vida das populações. Mesmo assim, contra tudo e contra todos, o Governo PSD/CDS-PP avançou com o processo de extinção de freguesias. E avançou, mesmo sabendo que são as freguesias os órgãos de poder que melhor conhecem a realidade social e que, além disso, têm a capacidade para chegar de forma mais direta àqueles que, efetivamente, contribuem para os cofres do Estado. O PSD e o CDS-PP, pretenderam, assim, poupar uns trocos — poupança, ainda assim, questionável —, à custa da qualidade de vida das populações e à custa do empobrecimento da nossa democracia. E, ao empobrecer a democracia, essa decisão impede os contribuintes de terem acesso direto aos órgãos de poder e aponta para uma gestão pública mais opaca e menos eficiente. Em bom rigor, esta lei de extinção de freguesias representa um inqualificável atentado à nossa democracia, à descentralização de poderes, ao desenvolvimento e à coesão social e territorial do País e está a fragilizar, de forma substancial, a prestação dos serviços públicos às populações, uma situação que importa de todo reverter de forma adequada e coerente. Ora, na perspetiva de Os Verdes, essa reversão deve passar não só por repor todas as freguesias extintas e cujos órgãos não se tenham pronunciado favoravelmente, mas também no caso das freguesias cujos órgãos deram parecer favorável, atribuindo a faculdade aos respetivos órgãos para poderem fazer uma avaliação do contributo dessa extinção para as populações e, se assim deliberarem, poderem comunicar à Assembleia da República a decisão no sentido da respetiva reposição. É esta a proposta que Os Verdes trazem para discussão, esperando agora, que volvido todos este tempo, esta Assembleia possa compreender os efeitos e as consequências que esta decisão representa para a nossa democracia e também para as populações.",
    "Sr. Presidente, Srs. Deputados: Tenho sempre alguma dificuldade em ouvir os argumentos do PSD para nunca votar nada do que o Bloco de Esquerda aqui traz em defesa dos professores e da carreira docente, porque, por cima das intervenções, oiço o eco de Rui Rio a dizer que há professores a mais e esse eco impede que eu consiga ouvir, depois, os argumentos que aqui dizem que há falta de professores e que é o Bloco de Esquerda que não faz aquilo que é preciso. De facto, houve um período em que se vincularam muitos professores na escola pública. Esse período foi aquele em que o Governo negociou com o Bloco de Esquerda duas vinculações extraordinárias e uma alteração à «norma-travão». Foi até 2018. A partir do momento em que o Governo «bateu com a porta» nessa negociação sobre a vinculação de professores, nunca mais foi possível dar um passo atrás na precariedade que existe quanto aos professores. Há mais de 18 000 professores precários contratados na escola pública. Quando a Sr.ª Deputada diz que este ano foram integrados 3000 professores para ajudar na compensação de alunos que tinham ficado para trás, isso não é verdade. As palavras enganam. Eles não foram integrados, foram contratados, o que quer dizer que há mais 3000 precários na escola pública. Nós sabemos que há problemas estruturais, há problemas de rejuvenescimento, há problemas de atratividade dos cursos superiores e há problemas de concursos, mas os partidos que aqui estão também nunca aceitaram as propostas que vieram dos únicos partidos que tiveram coragem para apresentar propostas sobre esses problemas.",
    "Sr. Presidente, cumprimento as Sr.as e os Srs. Deputados e começo por saudar os milhares de subscritores da petição que temos hoje em discussão: Discutimos a Petição n.º 59/XIV/1.ª, que tem como primeira peticionária a Sr.ª Márcia Alexandra Teixeira e tem mais de 34 000 assinaturas. Esta petição deu entrada na Comissão em abril de 2020 e os peticionários referem que os sócios-gerentes das pequenas empresas são eles próprios trabalhadores dessas organizações e ficaram excluídos do regime de layoff simplificado que foi criado para apoiar as empresas durante a pandemia originada pela COVID-19. Consideram digno e justo ser-lhes atribuída a isenção de impostos e segurança social e a atribuição de um valor mensal, correspondente ao salário mínimo nacional, pelo período que se estender a crise. Discutimos também o Projeto de Lei n.º 635/XIV/2.ª (CDS-PP) — Apoio aos sócios-gerentes e trabalhadores independentes. Tal como refere o CDS na exposição de motivos do seu projeto de lei, «como consequência da pandemia, pelas restrições de circulação e de prevenção sanitária, foram muitos os impactos nos mais diversos setores económicos, tendo sido sentidas dificuldades à produtividade e à sustentabilidade financeira dos negócios em todas as dimensões». No início deste ano, o Governo apresentou um pacote alargado de medidas que visam responder às necessidades das empresas, dos sócios-gerentes, dos trabalhadores independentes, nomeadamente: apoio extraordinário ao rendimento dos trabalhadores, medida proposta pelo Partido Socialista e aprovada no Orçamento do Estado para 2021, que tem por fim garantir a continuidade dos rendimentos das pessoas que se encontrem em situação de particular desproteção em virtude da COVID-19 e que se dirige a trabalhadores por conta de outrem, que terminem o subsídio de desemprego e o subsídio social de desemprego em 2021, e os trabalhadores independentes ou membros de órgãos estatutários que terminem o subsídio de cessação de atividade profissional em 2021 e cujas atividades estejam sujeitas ao dever de encerramento; apoio para trabalhadores por conta de outrem ou trabalhadores independentes economicamente dependentes que se encontrem em situação de desemprego involuntário, sem acesso a prestações de desemprego; apoio também a trabalhadores independentes ou trabalhadores de serviço doméstico com quebra de 40% de rendimentos e também aos gerentes de micro ou pequenas empresas e membros de órgãos estatutários de fundações, associações ou cooperativas com quebra de faturação de 40%. Este apoio vem garantir a proteção de trabalhadores e de gerentes e membros de órgãos estatutários, indo ao encontro, e até mais longe, do que o pretendido pelos peticionários. Outras medidas importantes para garantir a sustentabilidade dos trabalhadores e das empresas— importa salientá-lo neste local e momento — são: a prorrogação do apoio à retoma progressiva para microempresas e médias empresas, que contempla a redução dos montantes das contribuições sociais e a manutenção da retribuição integral dos trabalhadores sem esforço adicional das empresas; o apoio simplificado para microempresas com quebra de faturação superior a 25%, que pode ir até dois salários mínimos nacionais por trabalhador; a requalificação de desempregados pelo Programa Apoiar.pt; o alargamento e flexibilização do Programa Apoiar, nomeadamente para médias empresas e empresas com mais de 250 trabalhadores mas menos de 50 milhões de euros de faturação e empresários em nome individual sem contabilidade organizada; a redução de restrições em sede de capitais próprios; a aprovação de candidaturas de empresas com dívidas à segurança social e à Autoridade Tributária, desde que sujeita à condição de regularização; e a majoração do Programa Apoiar para estabelecimentos encerrados desde março. Importantes são ainda medidas de apoio ao arrendamento, à fiscalidade e ao financiamento das empresas, de que se destacam: o apoio a fundo perdido para o pagamento de rendas para empresas com quebra de faturação de 25% a 40% e também outro apoio para quebras superiores a 40%; a criação de linhas de crédito destinadas a inquilinos e senhorios; a criação de várias linhas de crédito para diversos setores de atividade, com uma componente a fundo perdido para atividades exportadoras e de eventos; o diferimento do IVA (imposto sobre o valor acrescentado) mensal e trimestral do primeiro semestre de 2021; a suspensão de execuções da Autoridade Tributária e segurança social, no primeiro semestre; a criação de um fundo de tesouraria para micro e pequenas empresas; o alargamento do microcrédito do Turismo de Portugal a pequenas empresas e o apoio à qualificação da oferta turística. Estas não são medidas avulsas, são medidas desenhadas para mitigar o impacto da crise na vida das empresas, dos empresários e dos trabalhadores, de forma tão abrangente quanto possível. Até lá, teremos de ser resilientes e ajustar, em cada dia, os nossos esforços e as nossas capacidades.",
    "Sr. Presidente, Srs. Deputados: O que tem acontecido com os sócios-gerentes é essencialmente uma falha por preconceito ideológico. É isso que tem acontecido. Nem sequer estamos a falar de falta de meios ou de falta de organização. É preconceito ideológico! Soubemos que muitos deste sócios-gerentes vão ter de devolver as isenções de layoff se pedirem o apoio à quebra de faturação, com retroativos, e já sabíamos disso quando chegámos a este debate, o que é a primeira das incoerências desta história. A segunda é os limites de faturação e o número de trabalhadores que o Governo socialista conseguiu impor a quem deveria ser apoiado como sócio-gerente. Basicamente, dissemos que «sim senhor, vamos apoiar», mas depois colocámos uma série de requisitos que, na prática, ou os obrigam a devolver os apoios ou os colocam fora da lista de apoios. São milhares, arrisco dizer centenas de milhares, aqueles que estão fora das listas de apoio do Partido Socialista. Mas era importante perceber também, hoje, por que é que o apoio à quebra de faturação, com retroativos, tão sido tão problemática, quando apontam o exemplo da Alemanha ou da Áustria, que, de facto, têm apostado nestas dimensões ao apoiar empresas que tenham tido quebra de faturação. Por que é que não são capazes de fazer exatamente a mesma coisa?!",
    "Sr. Presidente, Sr.as e Srs. Deputados: Gostava de começar a minha intervenção sobre os diversos projetos de criação de freguesia que hoje discutimos com uma palavra, que creio ser a palavra de ordem que preside, pelo menos, a alguns deles, restando saber se presidirá a todos. Essa palavra de ordem é reversão — reversão da reforma das freguesias, levada a cabo em 2013. Sobre essa palavra de ordem, gostava de dizer, para ser absolutamente clara e não haver equívocos, que o CDS está contra a reversão da reforma. Estamos perante uma reforma que, como qualquer reforma, não terá sido perfeita, mas que, sim, foi necessária e, sim, tem um balanço muito positivo, do nosso ponto de vista. Oito anos depois desta reforma, feita em 2013, a menos de um ano de eleições autárquicas, no meio de uma pandemia e numa crise social, sanitária e económica sem precedentes, o Governo escolhe este momento para voltar a colocar esta questão em cima da mesa. Portanto, é preciso dizer — e gostava de o dizer com clareza e sem tibiezas — que este é o pior momento de todos para termos esta discussão. Há muitas outras discussões mais urgentes e muitíssimo mais pertinentes e necessárias, neste momento. Em segundo lugar, há alguns projetos que são de reversão assumida — e nisso têm essa clareza —, e há a proposta de lei do Governo que pode ou não ser de reversão encapotada, algo que gostava de perceber. Mas vamos ao concreto: quando analisamos a proposta de lei, esta diz ter dois objetivos essenciais. Ora, em relação aos objetivos, estou de acordo, gostava era de obter alguns esclarecimentos quanto àquilo que eles são. Quando se diz que um dos objetivos é «melhorar a qualidade da democracia», estou absolutamente de acordo. Agora, se «melhorar a qualidade da democracia» implica aumentar o número de instituições e, necessariamente, aumentar o número de freguesias e aumentar o número de eleitos democraticamente nessas freguesias, o número de políticos eleitos, nesse caso, com esta visão, já não estou de acordo. Da mesma maneira, quando se diz «investir na qualidade dos serviços públicos», se isso significa aumentar a quantidade de serviços públicos, coisa que necessariamente acontece se houver um aumento muito significativo do número de freguesias, é bom que fique claro que, se aumentarmos significativamente o número de freguesias, o que estamos a fazer é desinvestir na qualidade dos serviços públicos, pois estamos a gastar mais com os eleitos e com as instituições do que estamos a gastar com os serviços. Portanto, gostava que me explicassem isto de uma maneira muito cristalina. Estamos a discutir projetos para elaborar uma lei-quadro de criação de freguesias que permita alguns ajustamentos pontuais, quer ao que aconteceu em 2013, quer sobretudo ao que vai acontecendo, porque a realidade, como é óbvio, não é estática e o território e as populações vão mudando, ou estamos aqui a discutir projetos para duplicar, triplicar ou quadruplicar o número de freguesias e voltarmos à situação passada? É que, se é para isso, sou radicalmente contra. Em terceiro lugar, e não menos importante, estamos a correr o risco de transformar as eleições autárquicas na pura e simples discussão do mapa autárquico e do mapa das freguesias. Por isso, pergunto se, aos discursos piedosos que sempre ouço contra o populismo e contra o radicalismo, queremos juntar a prática e evitar ter discussões destas em cima de eleições autárquicas. É que não há, de certeza absoluta, melhor incentivo ao populismo e à demagogia do que estarmos a ter esta discussão em cima de eleições autárquicas. Por último, gostava de lembrar o seguinte: a descentralização é um objetivo que todos partilhamos, mas é um objetivo que foi adiado. É que teoricamente, este ano — de acordo com o anterior Governo do Partido Socialista que dizia que esta era a grande reforma, em 2021 —, em janeiro deste ano, já teríamos, em velocidade de cruzeiro, a descentralização de competências para todos os municípios. Ora, em vez disso, o que sabemos é que, no que toca à saúde, à educação e à ação social, isso foi adiado para 2022. E, Sr.as e Srs. Deputados, foi adiado porque é nessas competências que estão as verbas e, mexer nessas competências, implica mexer na máquina do Estado central, e é exatamente isso que é sucessivamente adiado. Sem discutir isso e sem ir a esse cerne da questão, acho que, em relação a todas as outras questões, temos, naturalmente, abertura para mudanças pontuais, mas não temos abertura para reversões em massa de legislação sobre freguesias. E ia ficar tudo na mesma!",
    "Sr. Presidente, começo por cumprimentar os Srs. Secretários de Estado aqui presentes e as Sr.as e Srs. Deputados: Temos em discussão o Decreto-Lei n.º 92/2020, de 23 de outubro, quer altera o regime geral da gestão de resíduos. O que temos aqui é um diploma que tem gerado uma enorme discórdia e não tem, como se percebe neste mesmo debate, conseguido reunir qualquer consenso parlamentar. Tem também a voz crítica, e muito crítica, de vários municípios. E porquê? Porque temos aqui um aumento absolutamente desmesurado daquilo que é a taxa de resíduos. Vejamos: hoje, paga-se 11 €/t; a partir de janeiro passaremos de 11 €/t para 22 €/t; a seguir, a boa notícia que fica é que a partir de 2025 se passa a pagar 35 €/t. Ou seja, em apenas cinco anos triplica-se o valor dessa mesma taxa. E quem é que, ao final do dia, vai sair e vai ser penalizado? Como é evidente, são as famílias, são as empresas, são os consumidores, pois são eles que, efetivamente, ao final do dia, vão pagar e vão ter de suportar esta mesma fatura. Não é por acaso que a Associação Nacional de Municípios e a Área Metropolitana de Lisboa vieram exigir ou, melhor, vieram pedir ao Governo — estamos a falar, em muitos casos, de autarcas socialistas — para pôr travão, para revogar esta medida. Face a isto, o que é que o Governo fez? O Governo não ouviu nada nem ninguém, não quis ouvir coisa nenhuma, mantém a sua e continua em frente. Portanto, Sr.as e Srs. Deputados, para o CDS é absolutamente claro que não é através do aumento de taxas que se encontra uma solução para conduzir a novos comportamentos — não é, seguramente, por aí! Assim, isto vem revelar algo que é crítico neste mesmo Governo, e nesta área específica, que é a falta de orientação política. Há uma falta de orientação política em matéria ambiental. E o que é que temos deste Governo? Planos e mais planos, e agora vamos ter aqui mais umas quantas taxas, taxas estas que qualquer cidadão que olhe para elas percebe o exagero que existe aqui, passando de 11 € para 22 € e depois para 35 €. Portanto, no entender do CDS é preciso reduzir esta mesma taxa e exigir ao Governo que haja, sim, políticas com impacto ambiental. Como já aqui foi dito — e bem — neste mesmo debate, pergunto: qual é a pressa que existe neste dossier? Ele só terá aplicação mais tarde. Por que é que há toda esta urgência? Não se percebe, seguramente será até para agradar a Bruxelas, mas o que é certo é que nem os municípios, nem as famílias, nem as empresas, como é evidente, ficam satisfeitas, ficam tranquilas com o aumento desta taxa.",
]

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

sentenças_tokenizadas = []

for i in range(len(df["Falas"])):
    sentenças_tokenizadas.append(sent_tokenize(df["Falas"][i], language='portuguese'))
    ind = []
    for j in range(len(sentenças_tokenizadas[i])):
        if sentenças_tokenizadas[i][j][-3:] == "Sr." or sentenças_tokenizadas[i][j][-4:] == "Srs.":
            ind.append(j)
    for x in ind[::-1]:
        sentenças_tokenizadas[i][x] = sentenças_tokenizadas[i][x] + sentenças_tokenizadas[i][x + 1]
        del(sentenças_tokenizadas[i][x])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
len(sentenças_tokenizadas)

6

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

m2m100_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to(device)
m2m100_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="pt", tgt_lang="en")

In [ ]:
translated_list = []

for i in trange(len(sentenças_tokenizadas), desc="Fala Atual".format(i)):
    encoded = list(map(lambda x: m2m100_tokenizer(x, return_tensors="pt"), sentenças_tokenizadas[i]))
    generated_tokens = list(map(lambda x: m2m100_model.generate(**x, forced_bos_token_id=m2m100_tokenizer.get_lang_id("en")), encoded))
    translated = list(map(lambda x: m2m100_tokenizer.batch_decode(x, skip_special_tokens=True), generated_tokens))
    flat_translated = [sentence for embed_sentence in translated for sentence in embed_sentence]
    translated_list.append(" ".join(flat_translated))

Fala Atual: 100%|██████████| 6/6 [23:36<00:00, 236.15s/it]


In [ ]:
translated_list

['Deputies: After the PSD and the CDS-PP have approved the proposal of law of the Government Passos Coelho/Paulo Portas, which would give rise to Law No. 22/2012 and materialized by Law No. 11-A/2013, our democracy became substantially poor. In fact, despite the pompous designation “administrative reorganization of the territory of the freguesias”, this law did not do anything but impose the extinction of the freguesias, which was, by the other hand, the only intended goal, although not assumed by the Government PSD/CDS, with this so-called “administrative reorganization”. The contestation to the extinction process of freguesias has been, and is, public and notorious and extends to all people and to the most varied sectors, of course to the municipalities. He wants to use the word before he makes the closing intervention. If the answer is negative, there can be no questions. Deputies: After the PSD and the CDS-PP have approved the proposal of law of the Government Passos Coelho/Paulo P

In [ ]:
df["Falas"] = [
    "Sr. Presidente, Srs. Membros do Governo, Sr.as e Srs. Deputados: Depois de o PSD e de o CDS-PP terem aprovado a proposta de lei do Governo Passos Coelho/Paulo Portas, que viria dar origem à Lei n.º 22/2012 e materializada pela Lei n.º 11-A/2013, a nossa democracia ficou substancialmente mais pobre. Na verdade, apesar da pomposa designação «Reorganização administrativa do território das freguesias», esta lei mais não veio fazer do que impor a extinção de freguesias, que foi, aliás, o único objetivo pretendido, ainda que não assumido pelo Governo PSD/CDS, com esta dita «reorganização administrativa». A contestação ao processo de extinção de freguesias foi, e é, pública e notória e estende-se a toda a gente e aos mais variados setores, desde logo às autarquias. pretende usar da palavra antes dela fazer a intervenção de encerramento. Se a resposta for negativa, não pode haver perguntas. Sr. Presidente, Srs. Membros do Governo, Sr.as e Srs. Deputados: Depois de o PSD e de o CDS-PP terem aprovado a proposta de lei do Governo Passos Coelho/Paulo Portas, que viria dar origem à Lei n.º 22/2012 e materializada pela Lei n.º 11-A/2013, a nossa democracia ficou substancialmente mais pobre. Na verdade, apesar da pomposa designação «Reorganização administrativa do território das freguesias», esta lei mais não veio fazer do que impor a extinção de freguesias, que foi, aliás, o único objetivo pretendido, ainda que não assumido pelo Governo PSD/CDS, com esta dita «reorganização administrativa». A contestação ao processo de extinção de freguesias foi, e é, pública e notória e estende-se a toda a gente e aos mais variados setores, desde logo às autarquias. Foram centenas e centenas os e-mails de cidadãos, os ofícios de assembleias municipais e de assembleias de freguesia de todo o País que chegaram ao Grupo Parlamentar «Os Verdes» e, presumo, também aos restantes grupos parlamentares, dando conta da sua oposição à extinção de freguesias. Por várias vezes, milhares de autarcas de todo o País organizaram vigílias, manifestações e protestos contra este processo. A ANAFRE e a Associação Nacional de Municípios opuseram-se, de forma determinada, aos propósitos desta dita «reorganização administrativa». Os motivos desta generalizada contestação residem sobretudo, por um lado, no facto de o Governo de então não ter atribuído qualquer relevância à «pronúncia» das próprias autarquias sobre o assunto e, por outro lado, porque o que esteve na origem deste processo foram elementos estranhos à própria organização administrativa. De facto, o que motivou esta decisão não foram os interesses das populações, e cedo os portugueses e os autarcas perceberam que este processo nada tinha a ver com a preocupação em melhorar a qualidade de vida das pessoas. Esta reforma foi desenhada a pensar em tudo menos nas pessoas, nos seus problemas e nas suas preocupações. O propósito deste processo foi, exclusivamente, a redução do número de autarquias, mesmo que isso implicasse, como implicou e como continua a implicar, sacrifícios e mais dificuldades na vida das populações. Mesmo assim, contra tudo e contra todos, o Governo PSD/CDS-PP avançou com o processo de extinção de freguesias. E avançou, mesmo sabendo que são as freguesias os órgãos de poder que melhor conhecem a realidade social e que, além disso, têm a capacidade para chegar de forma mais direta àqueles que, efetivamente, contribuem para os cofres do Estado. O PSD e o CDS-PP, pretenderam, assim, poupar uns trocos — poupança, ainda assim, questionável —, à custa da qualidade de vida das populações e à custa do empobrecimento da nossa democracia. E, ao empobrecer a democracia, essa decisão impede os contribuintes de terem acesso direto aos órgãos de poder e aponta para uma gestão pública mais opaca e menos eficiente. Em bom rigor, esta lei de extinção de freguesias representa um inqualificável atentado à nossa democracia, à descentralização de poderes, ao desenvolvimento e à coesão social e territorial do País e está a fragilizar, de forma substancial, a prestação dos serviços públicos às populações, uma situação que importa de todo reverter de forma adequada e coerente. Ora, na perspetiva de Os Verdes, essa reversão deve passar não só por repor todas as freguesias extintas e cujos órgãos não se tenham pronunciado favoravelmente, mas também no caso das freguesias cujos órgãos deram parecer favorável, atribuindo a faculdade aos respetivos órgãos para poderem fazer uma avaliação do contributo dessa extinção para as populações e, se assim deliberarem, poderem comunicar à Assembleia da República a decisão no sentido da respetiva reposição. É esta a proposta que Os Verdes trazem para discussão, esperando agora, que volvido todos este tempo, esta Assembleia possa compreender os efeitos e as consequências que esta decisão representa para a nossa democracia e também para as populações.",
    "Sr. Presidente, Srs. Deputados: Tenho sempre alguma dificuldade em ouvir os argumentos do PSD para nunca votar nada do que o Bloco de Esquerda aqui traz em defesa dos professores e da carreira docente, porque, por cima das intervenções, oiço o eco de Rui Rio a dizer que há professores a mais e esse eco impede que eu consiga ouvir, depois, os argumentos que aqui dizem que há falta de professores e que é o Bloco de Esquerda que não faz aquilo que é preciso. De facto, houve um período em que se vincularam muitos professores na escola pública. Esse período foi aquele em que o Governo negociou com o Bloco de Esquerda duas vinculações extraordinárias e uma alteração à «norma-travão». Foi até 2018. A partir do momento em que o Governo «bateu com a porta» nessa negociação sobre a vinculação de professores, nunca mais foi possível dar um passo atrás na precariedade que existe quanto aos professores. Há mais de 18 000 professores precários contratados na escola pública. Quando a Sr.ª Deputada diz que este ano foram integrados 3000 professores para ajudar na compensação de alunos que tinham ficado para trás, isso não é verdade. As palavras enganam. Eles não foram integrados, foram contratados, o que quer dizer que há mais 3000 precários na escola pública. Nós sabemos que há problemas estruturais, há problemas de rejuvenescimento, há problemas de atratividade dos cursos superiores e há problemas de concursos, mas os partidos que aqui estão também nunca aceitaram as propostas que vieram dos únicos partidos que tiveram coragem para apresentar propostas sobre esses problemas.",
    "Sr. Presidente, cumprimento as Sr.as e os Srs. Deputados e começo por saudar os milhares de subscritores da petição que temos hoje em discussão: Discutimos a Petição n.º 59/XIV/1.ª, que tem como primeira peticionária a Sr.ª Márcia Alexandra Teixeira e tem mais de 34 000 assinaturas. Esta petição deu entrada na Comissão em abril de 2020 e os peticionários referem que os sócios-gerentes das pequenas empresas são eles próprios trabalhadores dessas organizações e ficaram excluídos do regime de layoff simplificado que foi criado para apoiar as empresas durante a pandemia originada pela COVID-19. Consideram digno e justo ser-lhes atribuída a isenção de impostos e segurança social e a atribuição de um valor mensal, correspondente ao salário mínimo nacional, pelo período que se estender a crise. Discutimos também o Projeto de Lei n.º 635/XIV/2.ª (CDS-PP) — Apoio aos sócios-gerentes e trabalhadores independentes. Tal como refere o CDS na exposição de motivos do seu projeto de lei, «como consequência da pandemia, pelas restrições de circulação e de prevenção sanitária, foram muitos os impactos nos mais diversos setores económicos, tendo sido sentidas dificuldades à produtividade e à sustentabilidade financeira dos negócios em todas as dimensões». No início deste ano, o Governo apresentou um pacote alargado de medidas que visam responder às necessidades das empresas, dos sócios-gerentes, dos trabalhadores independentes, nomeadamente: apoio extraordinário ao rendimento dos trabalhadores, medida proposta pelo Partido Socialista e aprovada no Orçamento do Estado para 2021, que tem por fim garantir a continuidade dos rendimentos das pessoas que se encontrem em situação de particular desproteção em virtude da COVID-19 e que se dirige a trabalhadores por conta de outrem, que terminem o subsídio de desemprego e o subsídio social de desemprego em 2021, e os trabalhadores independentes ou membros de órgãos estatutários que terminem o subsídio de cessação de atividade profissional em 2021 e cujas atividades estejam sujeitas ao dever de encerramento; apoio para trabalhadores por conta de outrem ou trabalhadores independentes economicamente dependentes que se encontrem em situação de desemprego involuntário, sem acesso a prestações de desemprego; apoio também a trabalhadores independentes ou trabalhadores de serviço doméstico com quebra de 40% de rendimentos e também aos gerentes de micro ou pequenas empresas e membros de órgãos estatutários de fundações, associações ou cooperativas com quebra de faturação de 40%. Este apoio vem garantir a proteção de trabalhadores e de gerentes e membros de órgãos estatutários, indo ao encontro, e até mais longe, do que o pretendido pelos peticionários. Outras medidas importantes para garantir a sustentabilidade dos trabalhadores e das empresas— importa salientá-lo neste local e momento — são: a prorrogação do apoio à retoma progressiva para microempresas e médias empresas, que contempla a redução dos montantes das contribuições sociais e a manutenção da retribuição integral dos trabalhadores sem esforço adicional das empresas; o apoio simplificado para microempresas com quebra de faturação superior a 25%, que pode ir até dois salários mínimos nacionais por trabalhador; a requalificação de desempregados pelo Programa Apoiar.pt; o alargamento e flexibilização do Programa Apoiar, nomeadamente para médias empresas e empresas com mais de 250 trabalhadores mas menos de 50 milhões de euros de faturação e empresários em nome individual sem contabilidade organizada; a redução de restrições em sede de capitais próprios; a aprovação de candidaturas de empresas com dívidas à segurança social e à Autoridade Tributária, desde que sujeita à condição de regularização; e a majoração do Programa Apoiar para estabelecimentos encerrados desde março. Importantes são ainda medidas de apoio ao arrendamento, à fiscalidade e ao financiamento das empresas, de que se destacam: o apoio a fundo perdido para o pagamento de rendas para empresas com quebra de faturação de 25% a 40% e também outro apoio para quebras superiores a 40%; a criação de linhas de crédito destinadas a inquilinos e senhorios; a criação de várias linhas de crédito para diversos setores de atividade, com uma componente a fundo perdido para atividades exportadoras e de eventos; o diferimento do IVA (imposto sobre o valor acrescentado) mensal e trimestral do primeiro semestre de 2021; a suspensão de execuções da Autoridade Tributária e segurança social, no primeiro semestre; a criação de um fundo de tesouraria para micro e pequenas empresas; o alargamento do microcrédito do Turismo de Portugal a pequenas empresas e o apoio à qualificação da oferta turística. Estas não são medidas avulsas, são medidas desenhadas para mitigar o impacto da crise na vida das empresas, dos empresários e dos trabalhadores, de forma tão abrangente quanto possível. Até lá, teremos de ser resilientes e ajustar, em cada dia, os nossos esforços e as nossas capacidades.",
    "Sr. Presidente, Srs. Deputados: O que tem acontecido com os sócios-gerentes é essencialmente uma falha por preconceito ideológico. É isso que tem acontecido. Nem sequer estamos a falar de falta de meios ou de falta de organização. É preconceito ideológico! Soubemos que muitos deste sócios-gerentes vão ter de devolver as isenções de layoff se pedirem o apoio à quebra de faturação, com retroativos, e já sabíamos disso quando chegámos a este debate, o que é a primeira das incoerências desta história. A segunda é os limites de faturação e o número de trabalhadores que o Governo socialista conseguiu impor a quem deveria ser apoiado como sócio-gerente. Basicamente, dissemos que «sim senhor, vamos apoiar», mas depois colocámos uma série de requisitos que, na prática, ou os obrigam a devolver os apoios ou os colocam fora da lista de apoios. São milhares, arrisco dizer centenas de milhares, aqueles que estão fora das listas de apoio do Partido Socialista. Mas era importante perceber também, hoje, por que é que o apoio à quebra de faturação, com retroativos, tão sido tão problemática, quando apontam o exemplo da Alemanha ou da Áustria, que, de facto, têm apostado nestas dimensões ao apoiar empresas que tenham tido quebra de faturação. Por que é que não são capazes de fazer exatamente a mesma coisa?!",
    "Sr. Presidente, Sr.as e Srs. Deputados: Gostava de começar a minha intervenção sobre os diversos projetos de criação de freguesia que hoje discutimos com uma palavra, que creio ser a palavra de ordem que preside, pelo menos, a alguns deles, restando saber se presidirá a todos. Essa palavra de ordem é reversão — reversão da reforma das freguesias, levada a cabo em 2013. Sobre essa palavra de ordem, gostava de dizer, para ser absolutamente clara e não haver equívocos, que o CDS está contra a reversão da reforma. Estamos perante uma reforma que, como qualquer reforma, não terá sido perfeita, mas que, sim, foi necessária e, sim, tem um balanço muito positivo, do nosso ponto de vista. Oito anos depois desta reforma, feita em 2013, a menos de um ano de eleições autárquicas, no meio de uma pandemia e numa crise social, sanitária e económica sem precedentes, o Governo escolhe este momento para voltar a colocar esta questão em cima da mesa. Portanto, é preciso dizer — e gostava de o dizer com clareza e sem tibiezas — que este é o pior momento de todos para termos esta discussão. Há muitas outras discussões mais urgentes e muitíssimo mais pertinentes e necessárias, neste momento. Em segundo lugar, há alguns projetos que são de reversão assumida — e nisso têm essa clareza —, e há a proposta de lei do Governo que pode ou não ser de reversão encapotada, algo que gostava de perceber. Mas vamos ao concreto: quando analisamos a proposta de lei, esta diz ter dois objetivos essenciais. Ora, em relação aos objetivos, estou de acordo, gostava era de obter alguns esclarecimentos quanto àquilo que eles são. Quando se diz que um dos objetivos é «melhorar a qualidade da democracia», estou absolutamente de acordo. Agora, se «melhorar a qualidade da democracia» implica aumentar o número de instituições e, necessariamente, aumentar o número de freguesias e aumentar o número de eleitos democraticamente nessas freguesias, o número de políticos eleitos, nesse caso, com esta visão, já não estou de acordo. Da mesma maneira, quando se diz «investir na qualidade dos serviços públicos», se isso significa aumentar a quantidade de serviços públicos, coisa que necessariamente acontece se houver um aumento muito significativo do número de freguesias, é bom que fique claro que, se aumentarmos significativamente o número de freguesias, o que estamos a fazer é desinvestir na qualidade dos serviços públicos, pois estamos a gastar mais com os eleitos e com as instituições do que estamos a gastar com os serviços. Portanto, gostava que me explicassem isto de uma maneira muito cristalina. Estamos a discutir projetos para elaborar uma lei-quadro de criação de freguesias que permita alguns ajustamentos pontuais, quer ao que aconteceu em 2013, quer sobretudo ao que vai acontecendo, porque a realidade, como é óbvio, não é estática e o território e as populações vão mudando, ou estamos aqui a discutir projetos para duplicar, triplicar ou quadruplicar o número de freguesias e voltarmos à situação passada? É que, se é para isso, sou radicalmente contra. Em terceiro lugar, e não menos importante, estamos a correr o risco de transformar as eleições autárquicas na pura e simples discussão do mapa autárquico e do mapa das freguesias. Por isso, pergunto se, aos discursos piedosos que sempre ouço contra o populismo e contra o radicalismo, queremos juntar a prática e evitar ter discussões destas em cima de eleições autárquicas. É que não há, de certeza absoluta, melhor incentivo ao populismo e à demagogia do que estarmos a ter esta discussão em cima de eleições autárquicas. Por último, gostava de lembrar o seguinte: a descentralização é um objetivo que todos partilhamos, mas é um objetivo que foi adiado. É que teoricamente, este ano — de acordo com o anterior Governo do Partido Socialista que dizia que esta era a grande reforma, em 2021 —, em janeiro deste ano, já teríamos, em velocidade de cruzeiro, a descentralização de competências para todos os municípios. Ora, em vez disso, o que sabemos é que, no que toca à saúde, à educação e à ação social, isso foi adiado para 2022. E, Sr.as e Srs. Deputados, foi adiado porque é nessas competências que estão as verbas e, mexer nessas competências, implica mexer na máquina do Estado central, e é exatamente isso que é sucessivamente adiado. Sem discutir isso e sem ir a esse cerne da questão, acho que, em relação a todas as outras questões, temos, naturalmente, abertura para mudanças pontuais, mas não temos abertura para reversões em massa de legislação sobre freguesias. E ia ficar tudo na mesma!",
    "Sr. Presidente, começo por cumprimentar os Srs. Secretários de Estado aqui presentes e as Sr.as e Srs. Deputados: Temos em discussão o Decreto-Lei n.º 92/2020, de 23 de outubro, quer altera o regime geral da gestão de resíduos. O que temos aqui é um diploma que tem gerado uma enorme discórdia e não tem, como se percebe neste mesmo debate, conseguido reunir qualquer consenso parlamentar. Tem também a voz crítica, e muito crítica, de vários municípios. E porquê? Porque temos aqui um aumento absolutamente desmesurado daquilo que é a taxa de resíduos. Vejamos: hoje, paga-se 11 €/t; a partir de janeiro passaremos de 11 €/t para 22 €/t; a seguir, a boa notícia que fica é que a partir de 2025 se passa a pagar 35 €/t. Ou seja, em apenas cinco anos triplica-se o valor dessa mesma taxa. E quem é que, ao final do dia, vai sair e vai ser penalizado? Como é evidente, são as famílias, são as empresas, são os consumidores, pois são eles que, efetivamente, ao final do dia, vão pagar e vão ter de suportar esta mesma fatura. Não é por acaso que a Associação Nacional de Municípios e a Área Metropolitana de Lisboa vieram exigir ou, melhor, vieram pedir ao Governo — estamos a falar, em muitos casos, de autarcas socialistas — para pôr travão, para revogar esta medida. Face a isto, o que é que o Governo fez? O Governo não ouviu nada nem ninguém, não quis ouvir coisa nenhuma, mantém a sua e continua em frente. Portanto, Sr.as e Srs. Deputados, para o CDS é absolutamente claro que não é através do aumento de taxas que se encontra uma solução para conduzir a novos comportamentos — não é, seguramente, por aí! Assim, isto vem revelar algo que é crítico neste mesmo Governo, e nesta área específica, que é a falta de orientação política. Há uma falta de orientação política em matéria ambiental. E o que é que temos deste Governo? Planos e mais planos, e agora vamos ter aqui mais umas quantas taxas, taxas estas que qualquer cidadão que olhe para elas percebe o exagero que existe aqui, passando de 11 € para 22 € e depois para 35 €. Portanto, no entender do CDS é preciso reduzir esta mesma taxa e exigir ao Governo que haja, sim, políticas com impacto ambiental. Como já aqui foi dito — e bem — neste mesmo debate, pergunto: qual é a pressa que existe neste dossier? Ele só terá aplicação mais tarde. Por que é que há toda esta urgência? Não se percebe, seguramente será até para agradar a Bruxelas, mas o que é certo é que nem os municípios, nem as famílias, nem as empresas, como é evidente, ficam satisfeitas, ficam tranquilas com o aumento desta taxa.",
]

In [ ]:
for i in range(len(df)):
    df["Sumarizado"][i] = df["Sumarizado"][i][0]

df

,Traduzido,Falas,Sumarizado
0,Deputies: After the PSD and the CDS-PP have ap...,"Sr. Presidente, Srs. Membros do Governo, Sr.as...","In fact, despite the pompous designation “admi..."
1,Deputies: I always have some difficulty in hea...,"Sr. Presidente, Srs. Deputados: Tenho sempre a...",Deputies: I always have some difficulty in hea...
2,Deputies and I start by greeting the thousands...,"Sr. Presidente, cumprimento as Sr.as e os Srs....",As the CDS points out in the exposure of the r...
3,Deputies: What has happened to the partners-ma...,"Sr. Presidente, Srs. Deputados: O que tem acon...",We knew that many of these managing partners w...
4,Deputies: I would like to start my interventio...,"Sr. Presidente, Sr.as e Srs. Deputados: Gostav...","Now, if “better the quality of democracy” impl..."
5,Deputies: We are discussing the Decree-Law No....,"Sr. Presidente, começo por cumprimentar os Srs...","We see: today, it is paid 11 /t; from January ..."


In [ ]:
df["Traduzido"] = translated_list

In [ ]:
df.to_csv("df.csv", index=False)

In [ ]:
df = pd.read_csv("df.csv")
df

,Traduzido,Falas
0,Deputies: After the PSD and the CDS-PP have ap...,"Sr. Presidente, Srs. Membros do Governo, Sr.as..."
1,Deputies: I always have some difficulty in hea...,"Sr. Presidente, Srs. Deputados: Tenho sempre a..."
2,Deputies and I start by greeting the thousands...,"Sr. Presidente, cumprimento as Sr.as e os Srs...."
3,Deputies: What has happened to the partners-ma...,"Sr. Presidente, Srs. Deputados: O que tem acon..."
4,Deputies: I would like to start my interventio...,"Sr. Presidente, Sr.as e Srs. Deputados: Gostav..."
5,Deputies: We are discussing the Decree-Law No....,"Sr. Presidente, começo por cumprimentar os Srs..."


In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
peg_model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large').to(device)
peg_tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')

In [ ]:
df["Sumarizado"] = df["Traduzido"].progress_apply(
    lambda x: [peg_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    for g in peg_model.generate(
        peg_tokenizer(x, return_tensors='pt').to(device)['input_ids'])
    ])
df.to_csv("df.csv", index=False)

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
df.to_csv("df.csv", index=False)

In [ ]:
df["Sumarizado"]

0    In fact, despite the pompous designation “admi...
1    Deputies: I always have some difficulty in hea...
2    As the CDS points out in the exposure of the r...
3    We knew that many of these managing partners w...
4    Now, if “better the quality of democracy” impl...
5    We see: today, it is paid 11 /t; from January ...
Name: Sumarizado, dtype: object

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

m2m100_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to(device)
m2m100_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="en", tgt_lang="pt")

df["Output"] = df["Sumarizado"].progress_apply(
                    lambda x:
                        m2m100_tokenizer.batch_decode(
                        m2m100_model.generate(
                            **m2m100_tokenizer(x,
                                return_tensors='pt',
                                max_length=2048,
                                truncation=True
                                ).to(device),
                            forced_bos_token_id=m2m100_tokenizer.get_lang_id("pt")
                            ),
                        skip_special_tokens=True
                        )
                    )

df.to_csv("df.csv", index=False)

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
for i in range(len(df)):
    df["Output"][i] = df["Output"][i][0]

df

,Traduzido,Falas,Sumarizado,Output
0,Deputies: After the PSD and the CDS-PP have ap...,"Sr. Presidente, Srs. Membros do Governo, Sr.as...","In fact, despite the pompous designation “admi...","De fato, apesar da designação pompous “reorgan..."
1,Deputies: I always have some difficulty in hea...,"Sr. Presidente, Srs. Deputados: Tenho sempre a...",Deputies: I always have some difficulty in hea...,Deputados: Eu sempre tenho alguma dificuldade ...
2,Deputies and I start by greeting the thousands...,"Sr. Presidente, cumprimento as Sr.as e os Srs....",As the CDS points out in the exposure of the r...,Como o CDS aponta na exposição das razões para...
3,Deputies: What has happened to the partners-ma...,"Sr. Presidente, Srs. Deputados: O que tem acon...",We knew that many of these managing partners w...,Sabíamos que muitos desses parceiros de gestão...
4,Deputies: I would like to start my interventio...,"Sr. Presidente, Sr.as e Srs. Deputados: Gostav...","Now, if “better the quality of democracy” impl...","Agora, se “melhor a qualidade da democracia” s..."
5,Deputies: We are discussing the Decree-Law No....,"Sr. Presidente, começo por cumprimentar os Srs...","We see: today, it is paid 11 /t; from January ...","Vemos: hoje, se paga 11 /t; a partir de janeir..."


In [ ]:
df.columns = ['Traduzido', 'Falas', 'Sumarizado', 'Output']

In [ ]:
df= df[['Falas', 'Traduzido', 'Sumarizado', 'Output']]
df

,Falas,Traduzido,Sumarizado,Output
0,"Sr. Presidente, Srs. Membros do Governo, Sr.as...",Deputies: After the PSD and the CDS-PP have ap...,"In fact, despite the pompous designation “admi...","De fato, apesar da designação pompous “reorgan..."
1,"Sr. Presidente, Srs. Deputados: Tenho sempre a...",Deputies: I always have some difficulty in hea...,Deputies: I always have some difficulty in hea...,Deputados: Eu sempre tenho alguma dificuldade ...
2,"Sr. Presidente, cumprimento as Sr.as e os Srs....",Deputies and I start by greeting the thousands...,As the CDS points out in the exposure of the r...,Como o CDS aponta na exposição das razões para...
3,"Sr. Presidente, Srs. Deputados: O que tem acon...",Deputies: What has happened to the partners-ma...,We knew that many of these managing partners w...,Sabíamos que muitos desses parceiros de gestão...
4,"Sr. Presidente, Sr.as e Srs. Deputados: Gostav...",Deputies: I would like to start my interventio...,"Now, if “better the quality of democracy” impl...","Agora, se “melhor a qualidade da democracia” s..."
5,"Sr. Presidente, começo por cumprimentar os Srs...",Deputies: We are discussing the Decree-Law No....,"We see: today, it is paid 11 /t; from January ...","Vemos: hoje, se paga 11 /t; a partir de janeir..."


In [ ]:
df.to_csv("df.csv", index=False)

In [ ]:
df.to_excel("df.xlsx", index=False)

In [ ]:
batch_2 = batch_2.progress_apply(
                    lambda x:
                        tokenizer.batch_decode(
                        model.generate(
                            **tokenizer(x,
                                return_tensors='pt',
                                max_length=2048,
                                truncation=True
                                ).to(device),
                            forced_bos_token_id=tokenizer.get_lang_id("en")
                            ),
                        skip_special_tokens=True
                        )
                    )

batch_2.to_csv(path='/content/drive/MyDrive/Iniciação Científica/Tradução/translated_df_2.csv', index=False)

  0%|          | 0/1130 [00:00<?, ?it/s]

,Falas,Traduzido,Sumarizado,Output
0,Deputies: After the PSD and the CDS-PP have ap...,"Sr. Presidente, Srs. Membros do Governo, Sr.as...","In fact, despite the pompous designation “admi...","De fato, apesar da designação pompous “reorgan..."
1,Deputies: I always have some difficulty in hea...,"Sr. Presidente, Srs. Deputados: Tenho sempre a...",Deputies: I always have some difficulty in hea...,Deputados: Eu sempre tenho alguma dificuldade ...
2,Deputies and I start by greeting the thousands...,"Sr. Presidente, cumprimento as Sr.as e os Srs....",As the CDS points out in the exposure of the r...,Como o CDS aponta na exposição das razões para...
3,Deputies: What has happened to the partners-ma...,"Sr. Presidente, Srs. Deputados: O que tem acon...",We knew that many of these managing partners w...,Sabíamos que muitos desses parceiros de gestão...
4,Deputies: I would like to start my interventio...,"Sr. Presidente, Sr.as e Srs. Deputados: Gostav...","Now, if “better the quality of democracy” impl...","Agora, se “melhor a qualidade da democracia” s..."
5,Deputies: We are discussing the Decree-Law No....,"Sr. Presidente, começo por cumprimentar os Srs...","We see: today, it is paid 11 /t; from January ...","Vemos: hoje, se paga 11 /t; a partir de janeir..."


In [ ]:
batch_3 = batch_3.progress_apply(
                    lambda x:
                        tokenizer.batch_decode(
                        model.generate(
                            **tokenizer(x,
                                return_tensors='pt',
                                max_length=2048,
                                truncation=True
                                ).to(device),
                            forced_bos_token_id=tokenizer.get_lang_id("en")
                            ),
                        skip_special_tokens=True
                        )
                    )

batch_3.to_csv(path='/content/drive/MyDrive/Iniciação Científica/Tradução/translated_df_3.csv', index=False)

In [ ]:
batch_4 = batch_4.progress_apply(
                    lambda x:
                        tokenizer.batch_decode(
                        model.generate(
                            **tokenizer(x,
                                return_tensors='pt',
                                max_length=2048,
                                truncation=True
                                ).to(device),
                            forced_bos_token_id=tokenizer.get_lang_id("en")
                            ),
                        skip_special_tokens=True
                        )
                    )

batch_4.to_csv(path='/content/drive/MyDrive/Iniciação Científica/Tradução/translated_df_4.csv', index=False)

In [ ]:
translated["Translated_5"] = batch_5.progress_apply(
                      lambda x:
                          tokenizer.batch_decode(
                          model.generate(
                              **tokenizer(x,
                                  return_tensors='pt',
                                  max_length=2048,
                                  truncation=True
                                  ).to(device),
                              forced_bos_token_id=tokenizer.get_lang_id("en")
                              ),
                          skip_special_tokens=True
                         )
                      )

translated.to_csv(path='/content/drive/MyDrive/Iniciação Científica/Tradução/translated_df_5.csv', index=False)

In [2]:
import pandas as pd

df = pd.read_csv("dialog_lines_database_initiatives.csv")

In [21]:
lens = []

for text in df["Text"].loc[df["Text"].str.contains("corrup")]:
    if len(text) == 2462 or len(text) == 2648:
        print(text)
        print("="*100 + '\n')

Sr. Presidente, ouvimos o Sr. Deputado António Filipe falar de política de justiça e dizer que não podemos ter uma justiça em que os cidadãos não podem confiar. É curioso ouvir isto do Partido Comunista, ouvir dizer que há vários arguidos e condenados em cargos públicos, quando, sempre que instado a comentar, diz que não se mistura justiça com política e que não confunde arguidos com condenados nem condenados com o exercício de cargos políticos. Por isso, seria bom que hoje ficasse claro o que querem aqui dizer! Protestos do Deputado do PCP Bruno Dias. Mas seria também importante perguntar ao Partido Comunista se está ou não disponível para conseguir superar o bloqueio do Tribunal Constitucional em matéria de enriquecimento ilícito e se está ou não disponível para uma proposta que, finalmente, consiga levar quem enriquece ilicitamente a ser condenado pela nossa justiça ou se é só de boca e se, na verdade, não vamos fazer nada. Mas hoje, Sr. Deputado, não posso deixar de lhe fazer uma p

In [22]:
"Sr. Presidente, ouvimos o Sr. Deputado António Filipe falar de política de justiça e dizer que não podemos ter uma justiça em que os cidadãos não podem confiar. É curioso ouvir isto do Partido Comunista, ouvir dizer que há vários arguidos e condenados em cargos públicos, quando, sempre que instado a comentar, diz que não se mistura justiça com política e que não confunde arguidos com condenados nem condenados com o exercício de cargos políticos. Por isso, seria bom que hoje ficasse claro o que querem aqui dizer! Protestos do Deputado do PCP Bruno Dias. Mas seria também importante perguntar ao Partido Comunista se está ou não disponível para conseguir superar o bloqueio do Tribunal Constitucional em matéria de enriquecimento ilícito e se está ou não disponível para uma proposta que, finalmente, consiga levar quem enriquece ilicitamente a ser condenado pela nossa justiça ou se é só de boca e se, na verdade, não vamos fazer nada. Mas hoje, Sr. Deputado, não posso deixar de lhe fazer uma pergunta. Sobre a prisão perpétua, ouvi o Sr. Deputado dizer que era um choque e que violava flagrantemente as nossas normas. Por isso, com esta oportunidade, tendo em conta que, na grande maioria dos países comunistas do mundo, há prisão perpétua — até há pior, mas há prisão perpétua, pelo menos —, gostaria que nos dissesse se o Partido Comunista Português vai ou não estar de acordo e é capaz de apoiar uma medida, como a que o Chega já introduziu ao sistema parlamentar português, de prisão perpétua para casos de violação de menores, homicídio, terrorismo e casos graves de corrupção, uma vez que em quase todos os países comunistas do mundo isso acontece. Gostaria ainda de lhe fazer outra pergunta, Sr. Deputado. O Sr. Deputado falou em incompatibilidade de exercício entre a vida pública e o setor privado, como se os privados fossem todos uns criminosos e uns corruptos. É tempo de o Partido Comunista dizer também se vai ou não apoiar a proposta do Chega que prevê que quem faça negócios com o Estado e em nome do Estado nunca mais possa exercer cargos na empresa com a qual negociou em nome do Estado. É que estamos fartos de ministros que fazem concessões e depois vão trabalhar para as pontes onde fizeram essas concessões ou paraministros que propuseram obras e que acabaram a trabalhar nas empresas a que adjudicaram essas mesmas obras. Era isto que o Partido Comunista podia fazer e era isto que, para lá da conversa sobre a política de justiça e sobre melhorar as celas individuais, como o Bloco de Esquerda quer fazer, os portugueses gostariam, efetivamente, de ver resolvido em Portugal. Protestos da Deputada do BE Isabel Pires."

'Sr. Presidente, ouvimos o Sr. Deputado António Filipe falar de política de justiça e dizer que não podemos ter uma justiça em que os cidadãos não podem confiar. É curioso ouvir isto do Partido Comunista, ouvir dizer que há vários arguidos e condenados em cargos públicos, quando, sempre que instado a comentar, diz que não se mistura justiça com política e que não confunde arguidos com condenados nem condenados com o exercício de cargos políticos. Por isso, seria bom que hoje ficasse claro o que querem aqui dizer! Protestos do Deputado do PCP Bruno Dias. Mas seria também importante perguntar ao Partido Comunista se está ou não disponível para conseguir superar o bloqueio do Tribunal Constitucional em matéria de enriquecimento ilícito e se está ou não disponível para uma proposta que, finalmente, consiga levar quem enriquece ilicitamente a ser condenado pela nossa justiça ou se é só de boca e se, na verdade, não vamos fazer nada. Mas hoje, Sr. Deputado, não posso deixar de lhe fazer uma 

In [24]:
sentences = [
        "Sr. Presidente, ouvimos o Sr. Deputado António Filipe falar de política de justiça e dizer que não podemos ter uma justiça em que os cidadãos não podem confiar.",
        "É curioso ouvir isto do Partido Comunista, ouvir dizer que há vários arguidos e condenados em cargos públicos, quando, sempre que instado a comentar, diz que não se mistura justiça com política e que não confunde arguidos com condenados nem condenados com o exercício de cargos políticos.",
        "Por isso, seria bom que hoje ficasse claro o que querem aqui dizer!",
        "Mas seria também importante perguntar ao Partido Comunista se está ou não disponível para conseguir superar o bloqueio do Tribunal Constitucional em matéria de enriquecimento ilícito e se está ou não disponível para uma proposta que, finalmente, consiga levar quem enriquece ilicitamente a ser condenado pela nossa justiça ou se é só de boca e se, na verdade, não vamos fazer nada.",
        "Mas hoje, Sr. Deputado, não posso deixar de lhe fazer uma pergunta.",
        "Sobre a prisão perpétua, ouvi o Sr. Deputado dizer que era um choque e que violava flagrantemente as nossas normas.",
        "Por isso, com esta oportunidade, tendo em conta que, na grande maioria dos países comunistas do mundo, há prisão perpétua — até há pior, mas há prisão perpétua, pelo menos —, gostaria que nos dissesse se o Partido Comunista Português vai ou não estar de acordo e é capaz de apoiar uma medida, como a que o Chega já introduziu ao sistema parlamentar português, de prisão perpétua para casos de violação de menores, homicídio, terrorismo e casos graves de corrupção, uma vez que em quase todos os países comunistas do mundo isso acontece.",
        "Gostaria ainda de lhe fazer outra pergunta, Sr. Deputado.",
        "O Sr. Deputado falou em incompatibilidade de exercício entre a vida pública e o setor privado, como se os privados fossem todos uns criminosos e uns corruptos.",
        "É tempo de o Partido Comunista dizer também se vai ou não apoiar a proposta do Chega que prevê que quem faça negócios com o Estado e em nome do Estado nunca mais possa exercer cargos na empresa com a qual negociou em nome do Estado.",
        "É que estamos fartos de ministros que fazem concessões e depois vão trabalhar para as pontes onde fizeram essas concessões ou paraministros que propuseram obras e que acabaram a trabalhar nas empresas a que adjudicaram essas mesmas obras.",
        "Era isto que o Partido Comunista podia fazer e era isto que, para lá da conversa sobre a política de justiça e sobre melhorar as celas individuais, como o Bloco de Esquerda quer fazer, os portugueses gostariam, efetivamente, de ver resolvido em Portugal.",
]

text = " ".join(sentences)

2565

In [27]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

m2m100_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="pt", tgt_lang="en")
m2m100_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

Downloading:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/272 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/908 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

In [28]:
encoded = list(map(lambda x: m2m100_tokenizer(x, return_tensors="pt"), sentences))
generated_tokens = list(map(lambda x: m2m100_model.generate(**x, forced_bos_token_id=m2m100_tokenizer.get_lang_id("en")), encoded))
translated = list(map(lambda x: m2m100_tokenizer.batch_decode(x, skip_special_tokens=True), generated_tokens))
flat_translated = [sentence for embed_sentence in translated for sentence in embed_sentence]
sent_translated = " ".join(flat_translated)

In [30]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

peg_model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large')
peg_tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')

Downloading:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

In [31]:
inputs = peg_tokenizer(sent_translated, return_tensors='pt')
summary_ids = peg_model.generate(inputs['input_ids'])
peg_output = [peg_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

In [34]:
len(peg_output[0])

1267

In [43]:
back = [
        "But it would also be important to ask the Communist Party whether or not it is available to overcome the blockade of the Constitutional Court in the matter of illegal enrichment and whether or not it is available for a proposal that ultimately can lead who illegally enrich to be condemned by our justice or whether it is just of mouth and whether, in fact, we will not do anything.",
        "Therefore, with this opportunity, having regard to the fact that in the vast majority of the communist countries of the world there is perpetual imprisonment — even worse, but there is perpetual imprisonment, at least — I would like you to tell us whether the Portuguese Communist Party will or will not agree and is able to support a measure, such as that Chega has already introduced to the Portuguese parliamentary system, of perpetual imprisonment for cases of minor violations, murder, terrorism and serious cases of corruption, since in almost all communist countries of the world this happens.",
        "It is time for the Communist Party to say whether or not to support the proposal of the Chega that provides that whoever does business with the State and in the name of the State can never again exercise positions in the company with which he has negotiated in the name of the State.",
]

In [44]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

m2m100_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="en", tgt_lang="pt")
m2m100_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

encoded = list(map(lambda x: m2m100_tokenizer(x, return_tensors="pt"), back))
generated_tokens = list(map(lambda x: m2m100_model.generate(**x, forced_bos_token_id=m2m100_tokenizer.get_lang_id("pt")), encoded))
translated = list(map(lambda x: m2m100_tokenizer.batch_decode(x, skip_special_tokens=True), generated_tokens))
flat_translated = [sentence for embed_sentence in translated for sentence in embed_sentence]
sent_translated = " ".join(flat_translated)

In [45]:
sent_translated

'Mas também seria importante perguntar ao Partido Comunista se está ou não disponível para superar o bloqueio do Tribunal Constitucional em matéria de enriquecimento ilegal e se está ou não disponível para uma proposta que finalmente possa levar quem ilegal enriquece a ser condenado pela nossa justiça ou se é simplesmente de boca e se, de fato, não faremos nada. Portanto, com esta oportunidade, tendo em conta o fato de que na grande maioria dos países comunistas do mundo há prisão perpétua – ainda pior, mas há prisão perpétua, pelo menos – eu gostaria de dizer-nos se o Partido Comunista Português vai ou não concordar e é capaz de apoiar uma medida, como que a Chega já introduziu no sistema parlamentar português, de prisão perpétua por casos de menores violações, assassinatos, terrorismo e sérios casos de corrupção, já que em quase todos os países comunistas do mundo isso acontece. É hora que o Partido Comunista diga se apoie ou não a proposta do Chega que prevê que quem faz negócios co

In [41]:
peg_output[0]

'But it would also be important to ask the Communist Party whether or not it is available to overcome the blockade of the Constitutional Court in the matter of illegal enrichment and whether or not it is available for a proposal that ultimately can lead who illegally enrich to be condemned by our justice or whether it is just of mouth and whether, in fact, we will not do anything. Therefore, with this opportunity, having regard to the fact that in the vast majority of the communist countries of the world there is perpetual imprisonment — even worse, but there is perpetual imprisonment, at least — I would like you to tell us whether the Portuguese Communist Party will or will not agree and is able to support a measure, such as that Chega has already introduced to the Portuguese parliamentary system, of perpetual imprisonment for cases of minor violations, murder, terrorism and serious cases of corruption, since in almost all communist countries of the world this happens. It is time for 

In [46]:
inputs = peg_tokenizer(peg_output[0], return_tensors='pt')
summary_ids = peg_model.generate(inputs['input_ids'])
peg_output_2 = [peg_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
peg_output_2[0]

'Therefore, with this opportunity, having regard to the fact that in the vast majority of the communist countries of the world there is perpetual imprisonment — even worse, but there is perpetual imprisonment, at least — I would like you to tell us whether the Portuguese Communist Party will or will not agree and is able to support a measure, such as that Chega has already introduced to the Portuguese parliamentary system, of perpetual imprisonment for cases of minor violations, murder, terrorism and serious cases of corruption, since in almost all communist countries of the world this happens.'

In [47]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

m2m100_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="en", tgt_lang="pt")
m2m100_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

encoded = list(map(lambda x: m2m100_tokenizer(x, return_tensors="pt"), peg_output_2))
generated_tokens = list(map(lambda x: m2m100_model.generate(**x, forced_bos_token_id=m2m100_tokenizer.get_lang_id("pt")), encoded))
translated = list(map(lambda x: m2m100_tokenizer.batch_decode(x, skip_special_tokens=True), generated_tokens))
flat_translated = [sentence for embed_sentence in translated for sentence in embed_sentence]
sent_translated = " ".join(flat_translated)

In [48]:
sent_translated

'Portanto, com esta oportunidade, tendo em conta o fato de que na grande maioria dos países comunistas do mundo há prisão perpétua – ainda pior, mas há prisão perpétua, pelo menos – eu gostaria de dizer-nos se o Partido Comunista Português vai ou não concordar e é capaz de apoiar uma medida, como que a Chega já introduziu no sistema parlamentar português, de prisão perpétua por casos de menores violações, assassinatos, terrorismo e sérios casos de corrupção, já que em quase todos os países comunistas do mundo isso acontece.'

In [56]:
len(sent_translated)

528

In [52]:
df.loc[df["Text"].str.contains("Partido Comunista s")]

,Unnamed: 0,Transcript,Date,Position,Person,Party,Text,Initiatives
1622,1622,DAR-007,1 DE OUTUBRO DE 2020,103,André Ventura,CH,"Sr. Presidente, ouvimos o Sr. Deputado António...",declarações políticas
7763,7763,DAR-033,23 DE DEZEMBRO DE 2020,37,André Ventura,CH,Mas o que fica deste debate é um Partido Comun...,unknown


In [57]:
df["Text"].iloc[1622]

'Sr. Presidente, ouvimos o Sr. Deputado António Filipe falar de política de justiça e dizer que não podemos ter uma justiça em que os cidadãos não podem confiar. É curioso ouvir isto do Partido Comunista, ouvir dizer que há vários arguidos e condenados em cargos públicos, quando, sempre que instado a comentar, diz que não se mistura justiça com política e que não confunde arguidos com condenados nem condenados com o exercício de cargos políticos. Por isso, seria bom que hoje ficasse claro o que querem aqui dizer! Protestos do Deputado do PCP Bruno Dias. Mas seria também importante perguntar ao Partido Comunista se está ou não disponível para conseguir superar o bloqueio do Tribunal Constitucional em matéria de enriquecimento ilícito e se está ou não disponível para uma proposta que, finalmente, consiga levar quem enriquece ilicitamente a ser condenado pela nossa justiça ou se é só de boca e se, na verdade, não vamos fazer nada. Mas hoje, Sr. Deputado, não posso deixar de lhe fazer uma 